In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from random import seed

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, BatchNormalization
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import models

Using TensorFlow backend.


In [2]:
#Generate Dataset
#Airfoil Self-Noise
df_airfoil=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Airfoil_self_noise.csv')

#Automobile
df_automobile=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Automobile_data.txt')

#Autompg
df_autompg=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Auto-mpg.csv')

#ChallengerUSA
df_challenger=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Challenger USA Space Shuttle O-Ring.csv')

#Combine Cycle Power
df_combine=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Combine Cycle Power Plant.csv')

#Computer Hardware
df_computer=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Computer Hardware.csv')

#Concrete Compressive Strength
df_concrete_compressive_strength=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Compressive Strength.csv')

#Concrete Slump Test
df_concrete1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete3=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')

#Energy Efficiency
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')

#Fertility Diagnosis
df_fertility=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\fertility_Diagnosis.csv')

#Forest Fires
df_forest=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\forestfires.csv')

#Housing
df_housing=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Housing.csv')

#Istanbul Stock Exchange
df_istanbul=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Istanbul Stock Exchange.csv')

#WDBC
df_wdbc=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\wdbc.csv')

In [3]:
#remove Nan columns of df_energy1
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy1.drop(df_energy1.columns[11],inplace=True, axis = 1)
df_energy1.drop(df_energy1.columns[10],inplace=True, axis = 1)

#remove Nan columns of df_energy2
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2.drop(df_energy2.columns[11],inplace=True, axis = 1)
df_energy2.drop(df_energy2.columns[10],inplace=True, axis = 1)

#remove question marks
#df_automobile
imp= Imputer(missing_values="NaN", strategy="mean")
df_automobile=df_automobile.replace("?", np.NaN)
df_automobile['num-of-doors'] = df_automobile['num-of-doors'].fillna("four")
for i in df_automobile.columns:
    if df_automobile[i].isnull().sum()>0:
        df_automobile[i]=imp.fit_transform(df_automobile[[i]])
#df_automobile = df_automobile[(df_automobile.astype(str) != '?').all(axis=1)]
#df_automobile.reset_index(drop=True, inplace=True)
df_automobile = df_automobile.drop(['engine-location'], axis=1)
#df_autompg
df_autompg = df_autompg[(df_autompg.astype(str) != '?').all(axis=1)]
df_autompg.reset_index(drop=True, inplace=True)

#convert string to number
le=LabelEncoder()
#df_automobile
string_automobile=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=string_automobile.apply(le.fit_transform)
encoder=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']].astype(np.int64)
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=encoder
a=df_automobile[['normalized-losses','horsepower','price']].astype(np.int64)
b=df_automobile[['bore','stroke','peak-rpm']].astype(float)
df_automobile[['normalized-losses','horsepower','price']]=a
df_automobile[['bore','stroke','peak-rpm']]=b
#df_computer
string_computer=df_computer[['Vendor','Model']]
df_computer[['Vendor','Model']]=string_computer.apply(le.fit_transform)
df_computer[['Vendor','Model']]=df_computer[['Vendor','Model']].astype(np.int64)

#dropna spesific column & special preprocessing
#df_forest
df_forest=df_forest[(df_forest['area'].astype(int) != 0)] 
df_forest.reset_index(drop=True, inplace=True)
num_features=10
dimension=(num_features+1)*9+10
population_size=(2*dimension+1)*2
df_forest=df_forest[:population_size]

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def preprocessing_data1(df,num_features):
    dimension=(num_features+1)*9+10
    population_size=(2*dimension+1)*2
    df=df.dropna()
    df=df[:population_size]
    df
    return df

In [5]:
def feature_selection(df):
    cor=df.corr()
    plt.figure(figsize=(100,50))
    sns.set(font_scale=3)
    sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
    plt.show()
    return cor

In [6]:
def train_test_split1(x, y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [7]:
def preprocessing_data2(df):
    train_size = 0.85*len(df)
    train_size = int(train_size)
    df_train = df[:train_size]
    df_test = df[train_size:]
    print(len(df_test))
    print(len(df_train))
    validation_size=len(df_test)/len(df_train)
    print(validation_size)
    return df_train, df_test, validation_size 

In [8]:
#df_airfoil=preprocessing_data1(df_airfoil,5)
#df_automobile=preprocessing_data1(df_automobile,22)
#df_autompg=preprocessing_data1(df_autompg,7)
#df_challenger=preprocessing_data1(df_challenger,3)
#df_combine=preprocessing_data1(df_combine,4)
#df_computer=preprocessing_data1(df_computer,9)
#df_concrete_compressive_strength=preprocessing_data1(df_concrete_compressive_strength,8)
#df_concrete1=preprocessing_data1(df_concrete1,7)
#df_concrete2=preprocessing_data1(df_concrete2,7)
#df_concrete3=preprocessing_data1(df_concrete3,7)
#df_energy1=preprocessing_data1(df_energy1,8)
#df_energy2=preprocessing_data1(df_energy2,8)

#df_fertility=preprocessing_data1(df_fertility,9)
for i in df_fertility:
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'N'] = 1
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'O'] = 2
df_fertility['Diagnosis']=df_fertility['Diagnosis'].astype(np.int64)

#df_forest=preprocessing_data1(df_forest,10)
#df_housing=preprocessing_data1(df_housing,13)

#df_istanbul=preprocessing_data1(df_istanbul,7)
new_header = df_istanbul.iloc[0]
df_istanbul = df_istanbul[1:]
df_istanbul.columns = new_header
df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']]=df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']].astype(np.float64)


#df_wdbc=preprocessing_data1(df_wdbc,10)
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a 

In [9]:
print(df_airfoil.head())
print(df_autompg.head())
print(df_automobile.head())
print(df_challenger.head())

   Frequency  Angle of Attack  Chord length  Free-stream velocity  \
0        800              0.0        0.3048                  71.3   
1       1000              0.0        0.3048                  71.3   
2       1250              0.0        0.3048                  71.3   
3       1600              0.0        0.3048                  71.3   
4       2000              0.0        0.3048                  71.3   

   Suction side displacement thickness  Scaled sound pressure level  
0                             0.002663                      126.201  
1                             0.002663                      125.201  
2                             0.002663                      125.951  
3                             0.002663                      127.591  
4                             0.002663                      127.461  
    Mpg  Cylinders  Displacement Horsepower  Weight  Acceleration  Model Year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  

In [10]:
#Train and Test 17 datasets
#Airfoil
x_airfoil = df_airfoil.drop(['Scaled sound pressure level'], axis = 1)
y_airfoil = df_airfoil['Scaled sound pressure level']
#Wdbc
x_wdbc = df_wdbc[['radius_mean','perimeter_mean','area_mean','concavity_mean','concave points_mean','radius_worst','perimeter_worst','area_worst','concave points_worst']]
y_wdbc = df_wdbc['diagnosis']
labelencoder_Y = LabelEncoder()
y_wdbc = labelencoder_Y.fit_transform(y_wdbc)
#Automobile
x_automobile = df_automobile.drop(['price','make','body-style'], axis = 1)
y_automobile = df_automobile['price']
y_automobile = np.reshape(y_automobile.values, (-1,1))
#Autompg
x_autompg = df_autompg.drop(['Mpg','Car Name'], axis = 1)
y_autompg = df_autompg['Mpg']
#Challenger
x_challenger = df_challenger.drop(['Number of O-rings','Temporal Order of Flight'], axis = 1)
y_challenger = df_challenger['Number of O-rings']
#Combine
x_combine = df_combine.drop(['PE'], axis = 1)
y_combine = df_combine['PE']
y_combine = np.reshape(y_combine.values, (-1,1))
#Computer
x_computer = df_computer.drop(['ERP'], axis = 1)
y_computer = df_computer['ERP']
y_computer = np.reshape(y_computer.values, (-1,1))
#Concrete_compressive_strength
x_concrete_compressive_strength = df_concrete_compressive_strength.drop(['CCS'], axis = 1)
y_concrete_compressive_strength = df_concrete_compressive_strength['CCS']
y_concrete_compressive_strength = np.reshape(y_concrete_compressive_strength.values, (-1,1))
#Concrete1
x_concrete1 = df_concrete1.drop(['No','Compressive Strength (28-day)(Mpa)'], axis = 1)
y_concrete1 = df_concrete1['Compressive Strength (28-day)(Mpa)']
#Concrete2
x_concrete2 = df_concrete2.drop(['No','SLUMP(cm)'], axis = 1)
y_concrete2 = df_concrete2['SLUMP(cm)']
y_concrete2 = np.reshape(y_concrete2.values, (-1,1))
#Concrete3
x_concrete3 = df_concrete3.drop(['No','FLOW(cm)'], axis = 1)
y_concrete3 = df_concrete3['FLOW(cm)']
y_concrete3 = np.reshape(y_concrete3.values, (-1,1))
#Energy1
x_energy1 = df_energy1.drop(['Y1'], axis = 1)
y_energy1 = df_energy1['Y1']
#Energy2
x_energy2 = df_energy2.drop(['Y2'], axis = 1)
y_energy2 = df_energy2['Y2']
#Fertility
x_fertility = df_fertility.drop(['Diagnosis'], axis = 1)
y_fertility = df_fertility['Diagnosis']
#Forest
x_forest = df_forest.drop(['month','day','area'], axis = 1)
y_forest = df_forest['area']
y_forest = np.reshape(y_forest.values, (-1,1))
#Housing
x_housing = df_housing.drop(['MEDV'], axis = 1)
y_housing = df_housing['MEDV']
#Istanbul
x_istanbul = df_istanbul.drop(['date','ISE1','ISE2'], axis = 1)
y_istanbul = df_istanbul[['ISE1','ISE2']]
y_istanbul['mean'] = y_istanbul.mean(axis=1)
y_istanbul = y_istanbul['mean']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil = train_test_split1(x_airfoil, y_airfoil)
x_train_automobile, x_test_automobile, y_train_automobile, y_test_automobile = train_test_split1(x_automobile, y_automobile)
x_train_autompg, x_test_autompg, y_train_autompg, y_test_autompg = train_test_split1(x_autompg, y_autompg)
x_train_challenger, x_test_challenger, y_train_challenger, y_test_challenger = train_test_split1(x_challenger, y_challenger)
x_train_combine, x_test_combine, y_train_combine, y_test_combine = train_test_split1(x_combine, y_combine)
x_train_computer, x_test_computer, y_train_computer, y_test_computer = train_test_split1(x_computer, y_computer)
x_train_concrete_compressive_strength, x_test_concrete_compressive_strength, y_train_concrete_compressive_strength, y_test_concrete_compressive_strength = train_test_split1(x_concrete_compressive_strength, y_concrete_compressive_strength)
x_train_concrete1, x_test_concrete1, y_train_concrete1, y_test_concrete1 = train_test_split1(x_concrete1, y_concrete1)
x_train_concrete2, x_test_concrete2, y_train_concrete2, y_test_concrete2 = train_test_split1(x_concrete2, y_concrete2)
x_train_concrete3, x_test_concrete3, y_train_concrete3, y_test_concrete3 = train_test_split1(x_concrete3, y_concrete3)
x_train_energy1, x_test_energy1, y_train_energy1, y_test_energy1 = train_test_split1(x_energy1, y_energy1)
x_train_energy2, x_test_energy2, y_train_energy2, y_test_energy2 = train_test_split1(x_energy2, y_energy2)
x_train_fertility, x_test_fertility, y_train_fertility, y_test_fertility = train_test_split1(x_fertility, y_fertility)
x_train_forest, x_test_forest, y_train_forest, y_test_forest = train_test_split1(x_forest, y_forest)
x_train_housing, x_test_housing, y_train_housing, y_test_housing = train_test_split1(x_housing, y_housing)
x_train_istanbul, x_test_istanbul, y_train_istanbul, y_test_istanbul = train_test_split1(x_istanbul, y_istanbul)
x_train_wdbc, x_test_wdbc, y_train_wdbc, y_test_wdbc = train_test_split1(x_wdbc, y_wdbc)

In [12]:
def train_val_split(x_train, y_train):
    X_train, X_val, Y_train, Y_val= train_test_split(x_train, y_train, test_size=0.176, random_state=42)
    return X_train, X_val, Y_train, Y_val

In [13]:
def standard_scaler(X_train, X_val, x_test):
    sc=StandardScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=sc.fit(X_train)
    #X_scale_val=sc.fit(X_val)
    #x_scale_test=sc.fit(x_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    return X_train, X_val, x_test, X_scale_train

In [14]:
def min_max_scaler_automobile(X_train, X_val, x_test, Y_train, Y_val, y_test):
    minmax_x=MinMaxScaler()
    minmax_y=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=minmax_x.fit(X_train)
    #X_scale_val=min_max.fit(X_val)
    #x_scale_test=min_max.fit(x_test)
    Y_scale_train=minmax_y.fit(Y_train)
    #Y_scale_val=min_max.fit(Y_val)
    #y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=Y_scale_train.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, X_scale_train, Y_scale_train

In [15]:
def min_max_scaler(X_train, X_val, x_test, Y_train, Y_val, y_test):
    min_max=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=min_max.fit(X_train)
    X_scale_val=min_max.fit(X_val)
    x_scale_test=min_max.fit(x_test)
    Y_scale_train=min_max.fit(Y_train)
    Y_scale_val=min_max.fit(Y_val)
    y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_val.transform(X_val)
    x_test=x_scale_test.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=y_scale_test.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, min_max

In [16]:
def normalization(X_train, X_val, x_test, Y_train, Y_val, y_test):
    normalizer=Normalizer()
    X_train=normalizer.fit_transform(X_train)
    X_val=normalizer.fit_transform(X_val)
    x_test=normalizer.fit_transform(x_test)
    Y_train=normalizer.fit_transform(Y_train)
    Y_val=normalizer.fit_transform(Y_val)
    y_test=normalizer.fit_transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test

In [17]:
#Optimizer list
adam0 = optimizers.Adam(lr=0.1)
adam1 = optimizers.Adam(lr=0.001)
adam2 = optimizers.Adam(lr=0.01)
adam3 = optimizers.Adam(lr=0.0001)
adam4 = optimizers.Adam(lr=0.00001)
sgd = optimizers.SGD(lr=0.00001, decay=1e-4, momentum=0.9, nesterov=True)
RMSprop= optimizers.RMSprop(lr=0.01,rho=0.9)
adagrad = optimizers.Adagrad(lr=0.01)
adadelta = optimizers.Adadelta(lr=1.0)

In [18]:
def NN_model_structure_regression_linear(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [19]:
def NN_model_structure_regression_relu(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('relu'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [20]:
def NN_model_structure_regression_sigmoid(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('sigmoid'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [21]:
def NN_model_structure_regression_tanh(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('tanh'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [22]:
X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil = train_val_split(x_train_airfoil, y_train_airfoil)
X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile = train_val_split(x_train_automobile, y_train_automobile)
X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg = train_val_split(x_train_autompg, y_train_autompg)
X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger = train_val_split(x_train_challenger, y_train_challenger)
X_train_combine, X_val_combine, Y_train_combine, Y_val_combine = train_val_split(x_train_combine, y_train_combine)
X_train_computer, X_val_computer, Y_train_computer, Y_val_computer = train_val_split(x_train_computer, y_train_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength = train_val_split(x_train_concrete_compressive_strength, y_train_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, Y_train_concrete1, Y_val_concrete1 = train_val_split(x_train_concrete1, y_train_concrete1)
X_train_concrete2, X_val_concrete2, Y_train_concrete2, Y_val_concrete2 = train_val_split(x_train_concrete2, y_train_concrete2)
X_train_concrete3, X_val_concrete3, Y_train_concrete3, Y_val_concrete3 = train_val_split(x_train_concrete3, y_train_concrete3)
X_train_energy1, X_val_energy1, Y_train_energy1, Y_val_energy1 = train_val_split(x_train_energy1, y_train_energy1)
X_train_energy2, X_val_energy2, Y_train_energy2, Y_val_energy2 = train_val_split(x_train_energy2, y_train_energy2)
X_train_fertility, X_val_fertility, Y_train_fertility, Y_val_fertility = train_val_split(x_train_fertility, y_train_fertility)
X_train_forest, X_val_forest, Y_train_forest, Y_val_forest = train_val_split(x_train_forest, y_train_forest)
X_train_housing, X_val_housing, Y_train_housing, Y_val_housing = train_val_split(x_train_housing, y_train_housing)
X_train_istanbul, X_val_istanbul, Y_train_istanbul, Y_val_istanbul = train_val_split(x_train_istanbul, y_train_istanbul)
X_train_wdbc, X_val_wdbc, Y_train_wdbc, Y_val_wdbc = train_val_split(x_train_wdbc, y_train_wdbc)

In [23]:
X_train_airfoil, X_val_airfoil, x_test_airfoil, X_scale_train_airfoil=standard_scaler(X_train_airfoil, X_val_airfoil, x_test_airfoil)
#X_train_automobile, X_val_automobile, x_test_automobile=standard_scaler(X_train_automobile, X_val_automobile, x_test_automobile)
#X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile = normalization(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile, X_scale_train_automobile, Y_scale_train_automobile=min_max_scaler_automobile(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_autompg, X_val_autompg, x_test_autompg, X_scale_train_autompg=standard_scaler(X_train_autompg, X_val_autompg, x_test_autompg)
X_train_challenger, X_val_challenger, x_test_challenger, X_scale_train_challenger=standard_scaler(X_train_challenger, X_val_challenger, x_test_challenger)
X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine, min_max_combine=min_max_scaler(X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine)
X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer, min_max_computer=min_max_scaler(X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength, min_max_concrete_compressive_strength=min_max_scaler(X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, x_test_concrete1, X_scale_train_concrete1=standard_scaler(X_train_concrete1, X_val_concrete1, x_test_concrete1)
X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2, min_max_concrete2=min_max_scaler(X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2)
X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3, min_max_concrete3=min_max_scaler(X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3)
X_train_energy1, X_val_energy1, x_test_energy1, X_scale_train_energy1=standard_scaler(X_train_energy1, X_val_energy1, x_test_energy1)
X_train_energy2, X_val_energy2, x_test_energy2, X_scale_train_energy2=standard_scaler(X_train_energy2, X_val_energy2, x_test_energy2)
X_train_fertility, X_val_fertility, x_test_fertility, X_scale_train_fertility=standard_scaler(X_train_fertility, X_val_fertility, x_test_fertility)
X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest, min_max_forest=min_max_scaler(X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest)
X_train_housing, X_val_housing, x_test_housing, X_scale_train_housing=standard_scaler(X_train_housing, X_val_housing, x_test_housing)
X_train_istanbul, X_val_istanbul, x_test_istanbul, X_scale_train_istanbul=standard_scaler(X_train_istanbul, X_val_istanbul, x_test_istanbul)

In [24]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_linear(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________

Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 424us/step - 

1052/1052 [==============================] - 0s 91us/step - loss: 21.6514 - val_loss: 26.1778
Epoch 66/1000
1052/1052 [==============================] - 0s 91us/step - loss: 21.9011 - val_loss: 25.2557
Epoch 67/1000
1052/1052 [==============================] - 0s 86us/step - loss: 21.9253 - val_loss: 26.8822
Epoch 68/1000
1052/1052 [==============================] - 0s 92us/step - loss: 21.8224 - val_loss: 25.6590
Epoch 69/1000
1052/1052 [==============================] - 0s 90us/step - loss: 22.2565 - val_loss: 25.3309
Epoch 70/1000
1052/1052 [==============================] - 0s 92us/step - loss: 22.3864 - val_loss: 26.4314
Epoch 71/1000
1052/1052 [==============================] - 0s 90us/step - loss: 22.1299 - val_loss: 26.0179
Epoch 72/1000
1052/1052 [==============================] - 0s 86us/step - loss: 21.5662 - val_loss: 26.1044
Epoch 73/1000
1052/1052 [==============================] - 0s 87us/step - loss: 21.5649 - val_loss: 25.6651
Epoch 74/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 91us/step - loss: 22.8110 - val_loss: 27.7081
Epoch 141/1000
1052/1052 [==============================] - 0s 89us/step - loss: 22.5989 - val_loss: 25.6591
Epoch 142/1000
1052/1052 [==============================] - 0s 89us/step - loss: 22.0785 - val_loss: 26.2292
Epoch 143/1000
1052/1052 [==============================] - 0s 86us/step - loss: 21.8308 - val_loss: 26.9076
Epoch 144/1000
1052/1052 [==============================] - 0s 96us/step - loss: 23.0201 - val_loss: 25.4238
Epoch 145/1000
1052/1052 [==============================] - 0s 90us/step - loss: 22.5811 - val_loss: 26.6687
Epoch 146/1000
1052/1052 [==============================] - 0s 88us/step - loss: 21.7068 - val_loss: 26.2613
Epoch 147/1000
1052/1052 [==============================] - 0s 89us/step - loss: 22.2354 - val_loss: 26.3618
Epoch 148/1000
1052/1052 [==============================] - 0s 88us/step - loss: 23.4323 - val_loss: 26.9516
Epoch 149/1000
1052/1052 [========

1052/1052 [==============================] - 0s 85us/step - loss: 18.5150 - val_loss: 23.6951
Epoch 216/1000
1052/1052 [==============================] - 0s 91us/step - loss: 18.8865 - val_loss: 22.6560
Epoch 217/1000
1052/1052 [==============================] - 0s 88us/step - loss: 18.5925 - val_loss: 22.3197
Epoch 218/1000
1052/1052 [==============================] - 0s 88us/step - loss: 18.5338 - val_loss: 22.2244
Epoch 219/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.8161 - val_loss: 22.3816
Epoch 220/1000
1052/1052 [==============================] - 0s 88us/step - loss: 18.4567 - val_loss: 23.1881
Epoch 221/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.9804 - val_loss: 21.5265
Epoch 222/1000
1052/1052 [==============================] - 0s 95us/step - loss: 18.0638 - val_loss: 20.9008
Epoch 223/1000
1052/1052 [==============================] - 0s 90us/step - loss: 18.1927 - val_loss: 21.1199
Epoch 224/1000
1052/1052 [========

1052/1052 [==============================] - 0s 89us/step - loss: 13.1902 - val_loss: 16.5702
Epoch 291/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.7659 - val_loss: 16.6561
Epoch 292/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.8350 - val_loss: 15.8790
Epoch 293/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.4029 - val_loss: 15.1843
Epoch 294/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.0623 - val_loss: 14.7333
Epoch 295/1000
1052/1052 [==============================] - 0s 85us/step - loss: 12.5594 - val_loss: 15.0017
Epoch 296/1000
1052/1052 [==============================] - 0s 89us/step - loss: 12.2040 - val_loss: 15.3397
Epoch 297/1000
1052/1052 [==============================] - 0s 87us/step - loss: 12.7113 - val_loss: 14.6781
Epoch 298/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.6923 - val_loss: 16.1626
Epoch 299/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 88us/step - loss: 8.2487 - val_loss: 9.4868
Epoch 746/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.6664 - val_loss: 9.4441
Epoch 747/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.4847 - val_loss: 9.9513
Epoch 748/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.5969 - val_loss: 9.9732
Epoch 749/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.1512 - val_loss: 9.7910
Epoch 750/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.0897 - val_loss: 9.4362
Epoch 751/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.1167 - val_loss: 10.1319
Epoch 752/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.1038 - val_loss: 9.5188
Epoch 753/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.1146 - val_loss: 10.6961
Epoch 754/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 91us/step - loss: 8.5682 - val_loss: 9.7284
Epoch 822/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.1240 - val_loss: 11.0201
Epoch 823/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.5475 - val_loss: 14.1508
Epoch 824/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.9016 - val_loss: 10.8664
Epoch 825/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.0439 - val_loss: 9.0589
Epoch 826/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.3292 - val_loss: 9.3244
Epoch 827/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.3532 - val_loss: 9.8085
Epoch 828/1000
1052/1052 [==============================] - 0s 100us/step - loss: 8.0949 - val_loss: 11.3893
Epoch 829/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.1998 - val_loss: 10.1649
Epoch 830/1000
1052/1052 [====================

1052/1052 [==============================] - 0s 88us/step - loss: 8.2969 - val_loss: 9.5112
Epoch 898/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.9904 - val_loss: 10.2019
Epoch 899/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.6091 - val_loss: 9.2266
Epoch 900/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.9495 - val_loss: 9.7043
Epoch 901/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.0636 - val_loss: 9.5831
Epoch 902/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.2075 - val_loss: 9.7070
Epoch 903/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.5188 - val_loss: 9.2588
Epoch 904/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.3295 - val_loss: 9.3678
Epoch 905/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.2819 - val_loss: 10.8230
Epoch 906/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 89us/step - loss: 8.7003 - val_loss: 9.2012
Epoch 974/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.7726 - val_loss: 9.3150
Epoch 975/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.3505 - val_loss: 9.4430
Epoch 976/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.4349 - val_loss: 10.3408
Epoch 977/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.8435 - val_loss: 10.5995
Epoch 978/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.1662 - val_loss: 9.5739
Epoch 979/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.0953 - val_loss: 10.0570
Epoch 980/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.3524 - val_loss: 11.1177
Epoch 981/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.4357 - val_loss: 9.6901
Epoch 982/1000
1052/1052 [======================

[array([[-0.1769592 , -1.9100463 , -3.710127  , -0.7330457 , -3.192411  ],
        [-0.18609002,  0.49548793,  0.18342125, -1.4196376 ,  0.07634293],
        [-0.2968387 ,  0.39592448, -0.70780814, -1.6240458 , -0.00972272],
        [ 0.1067272 ,  0.01967674,  0.14045534,  0.24968916,  0.0558002 ],
        [-0.15981774, -0.31168342, -0.31994483,  0.3565672 , -2.356783  ]],
       dtype=float32),
 array([ 1.3463533, -1.2876322, -4.6895347, -0.7310286, -4.682707 ],
       dtype=float32),
 array([[-1.2818673 , -1.7822881 , -1.2047614 , -1.5327519 ,  2.217907  ,
          2.0535457 ,  1.1480447 ,  2.0110862 , -1.2878984 , -1.0638355 ],
        [-0.04187602, -0.27881634, -0.81227756, -0.6709211 ,  0.98949784,
          0.57823056,  0.36546516,  0.25199446, -1.1619004 , -1.0453991 ],
        [ 2.437426  ,  1.4255244 ,  2.4320202 ,  2.3462694 , -2.458256  ,
         -2.0413668 , -2.2727752 , -2.5238566 ,  2.0347672 ,  2.1538606 ],
        [-0.5307615 , -0.22970738, -0.25091624,  0.3168216 ,  

In [25]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_linear_2nd.h5')

In [26]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_relu(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 199us/step - l

1052/1052 [==============================] - 0s 97us/step - loss: 18.6358 - val_loss: 24.2244
Epoch 68/1000
1052/1052 [==============================] - 0s 91us/step - loss: 19.0557 - val_loss: 23.8615
Epoch 69/1000
1052/1052 [==============================] - 0s 88us/step - loss: 19.7136 - val_loss: 24.0489
Epoch 70/1000
1052/1052 [==============================] - 0s 87us/step - loss: 18.7824 - val_loss: 24.6124
Epoch 71/1000
1052/1052 [==============================] - 0s 91us/step - loss: 18.6436 - val_loss: 23.0741
Epoch 72/1000
1052/1052 [==============================] - 0s 89us/step - loss: 18.8326 - val_loss: 22.9621
Epoch 73/1000
1052/1052 [==============================] - 0s 90us/step - loss: 18.6159 - val_loss: 23.2590
Epoch 74/1000
1052/1052 [==============================] - 0s 90us/step - loss: 18.8985 - val_loss: 24.0257
Epoch 75/1000
1052/1052 [==============================] - 0s 91us/step - loss: 18.3194 - val_loss: 22.8749
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 88us/step - loss: 18.5542 - val_loss: 22.8250
Epoch 143/1000
1052/1052 [==============================] - 0s 89us/step - loss: 18.2671 - val_loss: 23.4340
Epoch 144/1000
1052/1052 [==============================] - 0s 90us/step - loss: 18.1318 - val_loss: 22.7748
Epoch 145/1000
1052/1052 [==============================] - 0s 90us/step - loss: 18.1190 - val_loss: 22.8355
Epoch 146/1000
1052/1052 [==============================] - 0s 87us/step - loss: 18.6325 - val_loss: 24.2544
Epoch 147/1000
1052/1052 [==============================] - 0s 90us/step - loss: 18.2572 - val_loss: 22.8432
Epoch 148/1000
1052/1052 [==============================] - 0s 92us/step - loss: 18.2760 - val_loss: 22.9245
Epoch 149/1000
1052/1052 [==============================] - 0s 89us/step - loss: 18.3149 - val_loss: 24.8699
Epoch 150/1000
1052/1052 [==============================] - 0s 89us/step - loss: 18.7594 - val_loss: 23.4399
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 89us/step - loss: 17.7002 - val_loss: 21.9183
Epoch 218/1000
1052/1052 [==============================] - 0s 90us/step - loss: 18.4893 - val_loss: 21.9205
Epoch 219/1000
1052/1052 [==============================] - 0s 93us/step - loss: 18.5650 - val_loss: 22.6754
Epoch 220/1000
1052/1052 [==============================] - 0s 92us/step - loss: 17.8786 - val_loss: 23.0533
Epoch 221/1000
1052/1052 [==============================] - 0s 89us/step - loss: 19.1164 - val_loss: 22.2106
Epoch 222/1000
1052/1052 [==============================] - 0s 91us/step - loss: 18.9568 - val_loss: 22.1723
Epoch 223/1000
1052/1052 [==============================] - 0s 88us/step - loss: 18.3311 - val_loss: 22.6167
Epoch 224/1000
1052/1052 [==============================] - 0s 91us/step - loss: 17.9403 - val_loss: 22.6274
Epoch 225/1000
1052/1052 [==============================] - 0s 88us/step - loss: 18.3075 - val_loss: 22.2635
Epoch 226/1000
1052/1052 [========

1052/1052 [==============================] - 0s 94us/step - loss: 18.5643 - val_loss: 24.0551
Epoch 293/1000
1052/1052 [==============================] - 0s 90us/step - loss: 17.9643 - val_loss: 23.2770
Epoch 294/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.8719 - val_loss: 23.5711
Epoch 295/1000
1052/1052 [==============================] - 0s 92us/step - loss: 17.7030 - val_loss: 23.1065
Epoch 296/1000
1052/1052 [==============================] - 0s 95us/step - loss: 18.3067 - val_loss: 22.1823
Epoch 297/1000
1052/1052 [==============================] - 0s 91us/step - loss: 18.1524 - val_loss: 23.9762
Epoch 298/1000
1052/1052 [==============================] - 0s 92us/step - loss: 17.6878 - val_loss: 22.5465
Epoch 299/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.7455 - val_loss: 22.4115
Epoch 300/1000
1052/1052 [==============================] - 0s 89us/step - loss: 18.1767 - val_loss: 23.3954
Epoch 301/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 92us/step - loss: 17.5675 - val_loss: 23.1821
Epoch 743/1000
1052/1052 [==============================] - 0s 90us/step - loss: 17.7007 - val_loss: 22.7805
Epoch 744/1000
1052/1052 [==============================] - 0s 89us/step - loss: 18.0112 - val_loss: 21.8797
Epoch 745/1000
1052/1052 [==============================] - 0s 90us/step - loss: 16.8625 - val_loss: 22.3742
Epoch 746/1000
1052/1052 [==============================] - 0s 90us/step - loss: 17.4145 - val_loss: 23.5965
Epoch 747/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.3404 - val_loss: 23.4278
Epoch 748/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.5946 - val_loss: 23.8919
Epoch 749/1000
1052/1052 [==============================] - 0s 91us/step - loss: 17.3083 - val_loss: 24.2871
Epoch 750/1000
1052/1052 [==============================] - 0s 87us/step - loss: 18.3414 - val_loss: 22.5556
Epoch 751/1000
1052/1052 [========

1052/1052 [==============================] - 0s 95us/step - loss: 17.4764 - val_loss: 22.8529
Epoch 818/1000
1052/1052 [==============================] - 0s 113us/step - loss: 18.1712 - val_loss: 21.6859
Epoch 819/1000
1052/1052 [==============================] - 0s 96us/step - loss: 17.5169 - val_loss: 22.6874
Epoch 820/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.7923 - val_loss: 22.0918
Epoch 821/1000
1052/1052 [==============================] - 0s 100us/step - loss: 16.7489 - val_loss: 22.4654
Epoch 822/1000
1052/1052 [==============================] - 0s 96us/step - loss: 17.1944 - val_loss: 21.9340
Epoch 823/1000
1052/1052 [==============================] - 0s 96us/step - loss: 17.8595 - val_loss: 23.5922
Epoch 824/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.3628 - val_loss: 22.6145
Epoch 825/1000
1052/1052 [==============================] - 0s 86us/step - loss: 17.7726 - val_loss: 21.4965
Epoch 826/1000
1052/1052 [======

1052/1052 [==============================] - 0s 91us/step - loss: 18.6388 - val_loss: 21.7015
Epoch 893/1000
1052/1052 [==============================] - 0s 91us/step - loss: 17.2487 - val_loss: 23.5681
Epoch 894/1000
1052/1052 [==============================] - 0s 92us/step - loss: 17.0365 - val_loss: 23.0233
Epoch 895/1000
1052/1052 [==============================] - 0s 91us/step - loss: 16.9728 - val_loss: 23.8535
Epoch 896/1000
1052/1052 [==============================] - 0s 91us/step - loss: 16.9014 - val_loss: 22.3771
Epoch 897/1000
1052/1052 [==============================] - 0s 90us/step - loss: 17.6678 - val_loss: 23.2460
Epoch 898/1000
1052/1052 [==============================] - 0s 91us/step - loss: 16.9197 - val_loss: 22.2403
Epoch 899/1000
1052/1052 [==============================] - 0s 93us/step - loss: 17.0835 - val_loss: 22.7032
Epoch 900/1000
1052/1052 [==============================] - 0s 99us/step - loss: 17.3731 - val_loss: 23.0658
Epoch 901/1000
1052/1052 [========

1052/1052 [==============================] - 0s 94us/step - loss: 17.1450 - val_loss: 23.6430
Epoch 968/1000
1052/1052 [==============================] - 0s 92us/step - loss: 17.8959 - val_loss: 22.2434
Epoch 969/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.0625 - val_loss: 22.3017
Epoch 970/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.2499 - val_loss: 23.7993
Epoch 971/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.7019 - val_loss: 22.8869
Epoch 972/1000
1052/1052 [==============================] - 0s 91us/step - loss: 17.4929 - val_loss: 23.8098
Epoch 973/1000
1052/1052 [==============================] - 0s 88us/step - loss: 16.9738 - val_loss: 21.6176
Epoch 974/1000
1052/1052 [==============================] - 0s 84us/step - loss: 17.4449 - val_loss: 22.1052
Epoch 975/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.2457 - val_loss: 21.8374
Epoch 976/1000
1052/1052 [========

[array([[-0.5439516 , -0.2850951 , -0.5038556 , -0.01980588,  1.6049594 ],
        [-0.04670838, -0.09782287,  0.12158583, -0.2985098 , -0.5313407 ],
        [ 2.1282825 , -0.8228036 , -0.40089998,  0.7831243 , -0.7990883 ],
        [ 0.05432128,  0.4532102 , -0.8419722 ,  0.53309405, -0.07131425],
        [ 0.03138215,  0.08613851, -0.47052607, -0.11091782, -0.4379164 ]],
       dtype=float32),
 array([2.21736   , 4.1826305 , 3.0609474 , 1.4030826 , 0.23487292],
       dtype=float32),
 array([[-0.08854456,  0.59191865, -0.29357487, -0.947443  , -1.0523553 ,
          0.16253084,  1.2060856 ,  0.20450011,  0.8258223 ,  0.2002914 ],
        [-0.8074934 ,  1.7370238 , -0.36671808, -0.33099744, -1.1870208 ,
          0.99632233,  1.6797211 ,  1.5286862 ,  2.0566661 ,  1.373589  ],
        [-0.5327261 ,  1.0119091 , -0.4687797 , -0.4098936 , -0.36504132,
          0.8162868 ,  0.36756527,  0.35469282,  1.4260429 , -0.4051751 ],
        [-0.27847177,  0.7199743 , -0.78104734, -0.67232746, -

In [27]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_relu_2nd.h5')

In [28]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_sigmoid(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 228us/step - l

1052/1052 [==============================] - 0s 89us/step - loss: 17.8245 - val_loss: 21.1881
Epoch 67/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.7967 - val_loss: 21.4008
Epoch 68/1000
1052/1052 [==============================] - 0s 90us/step - loss: 17.5838 - val_loss: 21.5784
Epoch 69/1000
1052/1052 [==============================] - 0s 91us/step - loss: 17.5986 - val_loss: 21.2392
Epoch 70/1000
1052/1052 [==============================] - 0s 92us/step - loss: 17.3896 - val_loss: 20.6821
Epoch 71/1000
1052/1052 [==============================] - 0s 95us/step - loss: 17.4579 - val_loss: 20.8102
Epoch 72/1000
1052/1052 [==============================] - 0s 90us/step - loss: 17.1785 - val_loss: 21.0643
Epoch 73/1000
1052/1052 [==============================] - 0s 90us/step - loss: 17.0234 - val_loss: 20.5754
Epoch 74/1000
1052/1052 [==============================] - 0s 87us/step - loss: 16.8681 - val_loss: 20.2235
Epoch 75/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 102us/step - loss: 12.8319 - val_loss: 17.6616
Epoch 142/1000
1052/1052 [==============================] - 0s 100us/step - loss: 12.7904 - val_loss: 17.6051
Epoch 143/1000
1052/1052 [==============================] - 0s 106us/step - loss: 12.8244 - val_loss: 17.5875
Epoch 144/1000
1052/1052 [==============================] - 0s 101us/step - loss: 12.8211 - val_loss: 17.7079
Epoch 145/1000
1052/1052 [==============================] - 0s 95us/step - loss: 12.8399 - val_loss: 17.7634
Epoch 146/1000
1052/1052 [==============================] - 0s 87us/step - loss: 12.7483 - val_loss: 17.5310
Epoch 147/1000
1052/1052 [==============================] - 0s 90us/step - loss: 12.7869 - val_loss: 17.7231
Epoch 148/1000
1052/1052 [==============================] - 0s 89us/step - loss: 12.7351 - val_loss: 17.3294
Epoch 149/1000
1052/1052 [==============================] - 0s 85us/step - loss: 12.7408 - val_loss: 17.4870
Epoch 150/1000
1052/1052 [====

1052/1052 [==============================] - 0s 89us/step - loss: 12.0520 - val_loss: 17.1397
Epoch 217/1000
1052/1052 [==============================] - 0s 91us/step - loss: 11.9273 - val_loss: 17.0910
Epoch 218/1000
1052/1052 [==============================] - 0s 91us/step - loss: 12.0586 - val_loss: 16.8293
Epoch 219/1000
1052/1052 [==============================] - 0s 90us/step - loss: 11.9788 - val_loss: 17.2054
Epoch 220/1000
1052/1052 [==============================] - 0s 102us/step - loss: 11.8869 - val_loss: 16.9403
Epoch 221/1000
1052/1052 [==============================] - 0s 92us/step - loss: 11.9711 - val_loss: 17.0731
Epoch 222/1000
1052/1052 [==============================] - 0s 87us/step - loss: 11.9432 - val_loss: 16.7024
Epoch 223/1000
1052/1052 [==============================] - 0s 93us/step - loss: 11.9155 - val_loss: 16.9080
Epoch 224/1000
1052/1052 [==============================] - 0s 93us/step - loss: 11.8525 - val_loss: 16.7817
Epoch 225/1000
1052/1052 [=======

1052/1052 [==============================] - 0s 91us/step - loss: 11.6775 - val_loss: 16.7143
Epoch 292/1000
1052/1052 [==============================] - 0s 92us/step - loss: 11.7324 - val_loss: 18.0515
Epoch 293/1000
1052/1052 [==============================] - 0s 93us/step - loss: 11.7924 - val_loss: 16.9875
Epoch 294/1000
1052/1052 [==============================] - 0s 92us/step - loss: 11.6634 - val_loss: 17.0106
Epoch 295/1000
1052/1052 [==============================] - 0s 90us/step - loss: 11.6873 - val_loss: 17.0053
Epoch 296/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.6929 - val_loss: 17.8135
Epoch 297/1000
1052/1052 [==============================] - 0s 87us/step - loss: 11.6900 - val_loss: 16.8691
Epoch 298/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.6860 - val_loss: 16.9392
Epoch 299/1000
1052/1052 [==============================] - 0s 91us/step - loss: 11.6754 - val_loss: 17.0048
Epoch 300/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 88us/step - loss: 9.9856 - val_loss: 11.5927
Epoch 742/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.4364 - val_loss: 11.9916
Epoch 743/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.3557 - val_loss: 11.9925
Epoch 744/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.2254 - val_loss: 11.8938
Epoch 745/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.3579 - val_loss: 11.3736
Epoch 746/1000
1052/1052 [==============================] - 0s 94us/step - loss: 9.3565 - val_loss: 12.1222
Epoch 747/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.2683 - val_loss: 11.3922
Epoch 748/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.1336 - val_loss: 11.5489
Epoch 749/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.1808 - val_loss: 11.1097
Epoch 750/1000
1052/1052 [=================

Epoch 817/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.0149 - val_loss: 10.4693
Epoch 818/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.0412 - val_loss: 10.2457
Epoch 819/1000
1052/1052 [==============================] - 0s 91us/step - loss: 7.9451 - val_loss: 10.1023
Epoch 820/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.0295 - val_loss: 10.0267
Epoch 821/1000
1052/1052 [==============================] - 0s 93us/step - loss: 7.9252 - val_loss: 10.1173
Epoch 822/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.9081 - val_loss: 10.2038
Epoch 823/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.9270 - val_loss: 10.1193
Epoch 824/1000
1052/1052 [==============================] - 0s 102us/step - loss: 7.9082 - val_loss: 10.0476
Epoch 825/1000
1052/1052 [==============================] - 0s 91us/step - loss: 7.9264 - val_loss: 10.4241
Epoch 826/1000
1052/1052 [=

1052/1052 [==============================] - 0s 98us/step - loss: 7.7033 - val_loss: 9.7511
Epoch 894/1000
1052/1052 [==============================] - 0s 96us/step - loss: 7.6202 - val_loss: 9.6368
Epoch 895/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.6507 - val_loss: 10.2785
Epoch 896/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.6017 - val_loss: 9.9790
Epoch 897/1000
1052/1052 [==============================] - 0s 99us/step - loss: 7.5845 - val_loss: 10.0852
Epoch 898/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.7547 - val_loss: 10.0535
Epoch 899/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.0381 - val_loss: 10.4103
Epoch 900/1000
1052/1052 [==============================] - 0s 101us/step - loss: 7.6738 - val_loss: 9.9140
Epoch 901/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.6982 - val_loss: 9.7628
Epoch 902/1000
1052/1052 [=====================

1052/1052 [==============================] - 0s 98us/step - loss: 6.9790 - val_loss: 8.5993
Epoch 970/1000
1052/1052 [==============================] - 0s 91us/step - loss: 6.9632 - val_loss: 8.6568
Epoch 971/1000
1052/1052 [==============================] - 0s 91us/step - loss: 7.0486 - val_loss: 8.9641
Epoch 972/1000
1052/1052 [==============================] - 0s 98us/step - loss: 7.2187 - val_loss: 8.7001
Epoch 973/1000
1052/1052 [==============================] - 0s 92us/step - loss: 7.0448 - val_loss: 8.8437
Epoch 974/1000
1052/1052 [==============================] - 0s 96us/step - loss: 7.2171 - val_loss: 8.6281
Epoch 975/1000
1052/1052 [==============================] - 0s 96us/step - loss: 6.9548 - val_loss: 8.9330
Epoch 976/1000
1052/1052 [==============================] - 0s 130us/step - loss: 7.0374 - val_loss: 8.5722
Epoch 977/1000
1052/1052 [==============================] - 0s 100us/step - loss: 6.9613 - val_loss: 8.3476
Epoch 978/1000
1052/1052 [========================

[array([[ 0.6698604 ,  2.3367116 , -1.0199645 ,  2.2972414 ,  3.4574733 ],
        [ 1.8625656 , -0.7192344 , -0.50644684, -2.7564816 , -0.77488464],
        [ 3.017841  ,  1.7193338 ,  3.2250261 , -2.1390846 , -3.297612  ],
        [-0.12032692, -0.20894322,  0.14290702,  0.06873189, -0.06423615],
        [ 1.1598681 ,  0.15013571, -0.16314313,  0.66092056,  1.0038007 ]],
       dtype=float32),
 array([-1.1243118 ,  3.8461075 ,  3.2035608 ,  1.9705693 ,  0.19349001],
       dtype=float32),
 array([[ 2.7477982 , -0.3332051 , -1.4985319 ,  0.9098487 , -0.8314651 ,
         -0.46028247,  2.8709044 ,  5.217002  ,  4.4809093 ,  3.9020743 ],
        [-4.1530943 , -2.642728  ,  0.4352868 ,  4.1686516 ,  0.45212018,
         -1.2220286 ,  1.5131849 ,  0.9637588 , -1.5002705 ,  2.3621504 ],
        [ 5.579906  ,  1.1464422 ,  3.7274806 ,  5.201348  ,  3.128038  ,
          1.0689194 ,  3.844951  ,  2.6416163 ,  1.0576779 , -1.6738511 ],
        [ 1.5928547 ,  0.999633  , -2.4938486 , -3.532100

In [29]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_sigmoid_2nd.h5')

In [30]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_tanh(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_7 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_8 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 254us/step - l

1052/1052 [==============================] - 0s 81us/step - loss: 16.8964 - val_loss: 20.1765
Epoch 67/1000
1052/1052 [==============================] - 0s 89us/step - loss: 16.6299 - val_loss: 20.6119
Epoch 68/1000
1052/1052 [==============================] - 0s 84us/step - loss: 16.4949 - val_loss: 19.6876
Epoch 69/1000
1052/1052 [==============================] - 0s 82us/step - loss: 16.1888 - val_loss: 19.7177
Epoch 70/1000
1052/1052 [==============================] - 0s 86us/step - loss: 15.8348 - val_loss: 19.1090
Epoch 71/1000
1052/1052 [==============================] - 0s 83us/step - loss: 15.6658 - val_loss: 19.2549
Epoch 72/1000
1052/1052 [==============================] - 0s 82us/step - loss: 15.5135 - val_loss: 19.5901
Epoch 73/1000
1052/1052 [==============================] - 0s 85us/step - loss: 15.4001 - val_loss: 18.8764
Epoch 74/1000
1052/1052 [==============================] - 0s 87us/step - loss: 15.2333 - val_loss: 19.0774
Epoch 75/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 87us/step - loss: 13.6900 - val_loss: 19.1946
Epoch 142/1000
1052/1052 [==============================] - 0s 85us/step - loss: 13.8169 - val_loss: 17.5220
Epoch 143/1000
1052/1052 [==============================] - 0s 91us/step - loss: 13.8069 - val_loss: 17.9010
Epoch 144/1000
1052/1052 [==============================] - 0s 85us/step - loss: 13.6650 - val_loss: 18.2701
Epoch 145/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.0404 - val_loss: 17.3786
Epoch 146/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.8067 - val_loss: 17.3686
Epoch 147/1000
1052/1052 [==============================] - 0s 92us/step - loss: 13.5631 - val_loss: 17.5883
Epoch 148/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.6525 - val_loss: 17.1478
Epoch 149/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.6269 - val_loss: 17.2037
Epoch 150/1000
1052/1052 [========

1052/1052 [==============================] - 0s 87us/step - loss: 12.4794 - val_loss: 16.3814
Epoch 217/1000
1052/1052 [==============================] - 0s 89us/step - loss: 12.4028 - val_loss: 16.4536
Epoch 218/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.3695 - val_loss: 15.9548
Epoch 219/1000
1052/1052 [==============================] - 0s 83us/step - loss: 12.3363 - val_loss: 17.1955
Epoch 220/1000
1052/1052 [==============================] - 0s 90us/step - loss: 12.4396 - val_loss: 16.2918
Epoch 221/1000
1052/1052 [==============================] - 0s 87us/step - loss: 12.3551 - val_loss: 16.6973
Epoch 222/1000
1052/1052 [==============================] - 0s 85us/step - loss: 12.0470 - val_loss: 13.4582
Epoch 223/1000
1052/1052 [==============================] - 0s 84us/step - loss: 10.7072 - val_loss: 12.6738
Epoch 224/1000
1052/1052 [==============================] - 0s 82us/step - loss: 10.0968 - val_loss: 16.4801
Epoch 225/1000
1052/1052 [========

1052/1052 [==============================] - 0s 98us/step - loss: 6.2169 - val_loss: 7.8814
Epoch 293/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.4601 - val_loss: 7.7441
Epoch 294/1000
1052/1052 [==============================] - 0s 100us/step - loss: 6.4025 - val_loss: 7.5618
Epoch 295/1000
1052/1052 [==============================] - 0s 100us/step - loss: 6.0783 - val_loss: 7.5790
Epoch 296/1000
1052/1052 [==============================] - 0s 94us/step - loss: 6.2932 - val_loss: 8.5826
Epoch 297/1000
1052/1052 [==============================] - 0s 92us/step - loss: 6.1300 - val_loss: 7.6450
Epoch 298/1000
1052/1052 [==============================] - 0s 81us/step - loss: 6.2759 - val_loss: 7.2775
Epoch 299/1000
1052/1052 [==============================] - 0s 84us/step - loss: 6.0122 - val_loss: 7.7728
Epoch 300/1000
1052/1052 [==============================] - 0s 84us/step - loss: 6.3432 - val_loss: 7.2992
Epoch 301/1000
1052/1052 [========================

...

1052/1052 [==============================] - 0s 82us/step - loss: 5.6983 - val_loss: 6.5832
Epoch 749/1000
1052/1052 [==============================] - 0s 83us/step - loss: 5.7749 - val_loss: 6.6203
Epoch 750/1000
1052/1052 [==============================] - 0s 77us/step - loss: 5.7367 - val_loss: 6.5111
Epoch 751/1000
1052/1052 [==============================] - 0s 100us/step - loss: 5.4705 - val_loss: 6.2516
Epoch 752/1000
1052/1052 [==============================] - 0s 98us/step - loss: 5.4912 - val_loss: 7.0868
Epoch 753/1000
1052/1052 [==============================] - 0s 109us/step - loss: 5.5313 - val_loss: 6.3801
Epoch 754/1000
1052/1052 [==============================] - 0s 98us/step - loss: 5.6897 - val_loss: 7.4034
Epoch 755/1000
1052/1052 [==============================] - 0s 82us/step - loss: 5.9475 - val_loss: 6.5275
Epoch 756/1000
1052/1052 [==============================] - 0s 95us/step - loss: 5.7591 - val_loss: 6.3778
Epoch 757/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 89us/step - loss: 5.5216 - val_loss: 6.9397
Epoch 825/1000
1052/1052 [==============================] - 0s 86us/step - loss: 6.0046 - val_loss: 6.2524
Epoch 826/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.6490 - val_loss: 6.7800
Epoch 827/1000
1052/1052 [==============================] - 0s 82us/step - loss: 5.6889 - val_loss: 6.4676
Epoch 828/1000
1052/1052 [==============================] - 0s 83us/step - loss: 5.5948 - val_loss: 6.7439
Epoch 829/1000
1052/1052 [==============================] - 0s 84us/step - loss: 5.8100 - val_loss: 6.3932
Epoch 830/1000
1052/1052 [==============================] - 0s 100us/step - loss: 5.7299 - val_loss: 7.3992
Epoch 831/1000
1052/1052 [==============================] - 0s 100us/step - loss: 5.9612 - val_loss: 6.6350
Epoch 832/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.4704 - val_loss: 6.8186
Epoch 833/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 85us/step - loss: 5.8845 - val_loss: 6.9284
Epoch 901/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.5884 - val_loss: 6.6299
Epoch 902/1000
1052/1052 [==============================] - 0s 83us/step - loss: 5.4612 - val_loss: 6.3715
Epoch 903/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.4490 - val_loss: 6.4567
Epoch 904/1000
1052/1052 [==============================] - 0s 84us/step - loss: 5.7959 - val_loss: 6.3531
Epoch 905/1000
1052/1052 [==============================] - 0s 83us/step - loss: 5.6682 - val_loss: 7.4956
Epoch 906/1000
1052/1052 [==============================] - 0s 84us/step - loss: 5.5623 - val_loss: 6.6894
Epoch 907/1000
1052/1052 [==============================] - 0s 82us/step - loss: 5.8692 - val_loss: 6.7638
Epoch 908/1000
1052/1052 [==============================] - 0s 83us/step - loss: 5.6816 - val_loss: 6.3961
Epoch 909/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 87us/step - loss: 5.7066 - val_loss: 6.6589
Epoch 977/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.7468 - val_loss: 6.6695
Epoch 978/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.4992 - val_loss: 6.8166
Epoch 979/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.5988 - val_loss: 6.9244
Epoch 980/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.6817 - val_loss: 6.6567
Epoch 981/1000
1052/1052 [==============================] - 0s 79us/step - loss: 5.7016 - val_loss: 6.9716
Epoch 982/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.6499 - val_loss: 6.5558
Epoch 983/1000
1052/1052 [==============================] - 0s 85us/step - loss: 6.0207 - val_loss: 6.5080
Epoch 984/1000
1052/1052 [==============================] - 0s 80us/step - loss: 5.9181 - val_loss: 6.7509
Epoch 985/1000
1052/1052 [==========================

[array([[-4.4630461e+00,  4.9712732e-01, -1.2574962e+00,  1.1052965e+00,
         -3.3215189e+00],
        [ 3.3546504e-01, -6.4337105e-02, -1.5781964e+00,  8.8481653e-01,
         -1.1144971e+00],
        [-1.3360842e-01, -2.6173387e+00, -1.6948987e+00,  3.2664686e-01,
         -6.4817256e-01],
        [-1.4254083e-01,  3.3331582e-01,  3.4243137e-01, -2.4920966e-01,
          4.9856105e-01],
        [ 4.8959276e-01,  5.5950022e-01, -1.3957974e-01,  9.7747308e-01,
         -3.8608964e-04]], dtype=float32),
 array([-0.6172113 , -0.8928102 , -0.46716642,  2.039005  , -1.0738779 ],
       dtype=float32),
 array([[-0.31958768,  0.3194537 ,  1.3429111 ,  0.3177818 ,  0.3204141 ,
         -0.3196611 ,  0.31349212,  0.19467573, -1.8852876 , -1.1351919 ],
        [-2.9312623 ,  2.9342828 ,  0.33660072,  2.9353313 , -0.19023427,
         -2.9306366 ,  3.0055728 ,  0.3215589 , -1.4263799 ,  2.1770751 ],
        [-0.20935294,  0.20721328,  0.18921827,  0.20492558,  0.3082143 ,
         -0.2097745

In [31]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_tanh_2nd.h5')

In [32]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_linear(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_9 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_10 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 683us/step - loss:

274/274 [==============================] - 0s 97us/step - loss: 6.9310 - val_loss: 8.2102
Epoch 70/1000
274/274 [==============================] - 0s 106us/step - loss: 7.0412 - val_loss: 8.7251
Epoch 71/1000
274/274 [==============================] - 0s 104us/step - loss: 7.0518 - val_loss: 8.3560
Epoch 72/1000
274/274 [==============================] - 0s 94us/step - loss: 6.7950 - val_loss: 8.5791
Epoch 73/1000
274/274 [==============================] - 0s 95us/step - loss: 6.7407 - val_loss: 8.5847
Epoch 74/1000
274/274 [==============================] - 0s 91us/step - loss: 6.8961 - val_loss: 8.3042
Epoch 75/1000
274/274 [==============================] - 0s 91us/step - loss: 6.7189 - val_loss: 8.4078
Epoch 76/1000
274/274 [==============================] - 0s 95us/step - loss: 6.8752 - val_loss: 8.4941
Epoch 77/1000
274/274 [==============================] - 0s 97us/step - loss: 6.6761 - val_loss: 8.2647
Epoch 78/1000
274/274 [==============================] - 0s 98us/step - loss

274/274 [==============================] - 0s 95us/step - loss: 6.2012 - val_loss: 8.3237
Epoch 148/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2840 - val_loss: 8.4351
Epoch 149/1000
274/274 [==============================] - 0s 91us/step - loss: 6.2217 - val_loss: 8.3225
Epoch 150/1000
274/274 [==============================] - 0s 97us/step - loss: 6.1668 - val_loss: 8.3846
Epoch 151/1000
274/274 [==============================] - 0s 91us/step - loss: 6.1802 - val_loss: 8.3307
Epoch 152/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3387 - val_loss: 8.5661
Epoch 153/1000
274/274 [==============================] - 0s 93us/step - loss: 6.4690 - val_loss: 8.5033
Epoch 154/1000
274/274 [==============================] - 0s 106us/step - loss: 6.3408 - val_loss: 8.4467
Epoch 155/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2364 - val_loss: 8.5267
Epoch 156/1000
274/274 [==============================] - 0s 94us/st

274/274 [==============================] - 0s 95us/step - loss: 5.7454 - val_loss: 8.4034
Epoch 226/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6034 - val_loss: 8.5010
Epoch 227/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6585 - val_loss: 8.4547
Epoch 228/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6389 - val_loss: 8.5135
Epoch 229/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6361 - val_loss: 8.5601
Epoch 230/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5616 - val_loss: 8.4007
Epoch 231/1000
274/274 [==============================] - 0s 95us/step - loss: 5.5968 - val_loss: 8.4202
Epoch 232/1000
274/274 [==============================] - 0s 91us/step - loss: 5.5504 - val_loss: 8.4576
Epoch 233/1000
274/274 [==============================] - 0s 96us/step - loss: 5.6143 - val_loss: 8.2811
Epoch 234/1000
274/274 [==============================] - 0s 99us/step

274/274 [==============================] - 0s 91us/step - loss: 5.4301 - val_loss: 8.3546
Epoch 304/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5807 - val_loss: 8.2199
Epoch 305/1000
274/274 [==============================] - 0s 87us/step - loss: 5.5883 - val_loss: 8.2670
Epoch 306/1000
274/274 [==============================] - 0s 91us/step - loss: 5.4856 - val_loss: 8.3841
Epoch 307/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4120 - val_loss: 8.4086
Epoch 308/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4844 - val_loss: 8.4286
Epoch 309/1000
274/274 [==============================] - 0s 95us/step - loss: 5.5798 - val_loss: 8.5621
Epoch 310/1000
274/274 [==============================] - 0s 90us/step - loss: 5.5348 - val_loss: 8.4089
Epoch 311/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4349 - val_loss: 8.2885
Epoch 312/1000
274/274 [==============================] - 0s 92us/ste

...

274/274 [==============================] - 0s 98us/step - loss: 5.1701 - val_loss: 7.9675
Epoch 693/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2043 - val_loss: 7.9870
Epoch 694/1000
274/274 [==============================] - 0s 107us/step - loss: 5.2250 - val_loss: 7.9846
Epoch 695/1000
274/274 [==============================] - 0s 96us/step - loss: 5.2850 - val_loss: 8.0695
Epoch 696/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3017 - val_loss: 7.8461
Epoch 697/1000
274/274 [==============================] - 0s 101us/step - loss: 5.1550 - val_loss: 7.9734
Epoch 698/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3949 - val_loss: 7.7523
Epoch 699/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3355 - val_loss: 8.1918
Epoch 700/1000
274/274 [==============================] - 0s 96us/step - loss: 5.4001 - val_loss: 7.8753
Epoch 701/1000
274/274 [==============================] - 0s 98us/st

274/274 [==============================] - 0s 95us/step - loss: 5.2934 - val_loss: 7.9735
Epoch 771/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2006 - val_loss: 7.8964
Epoch 772/1000
274/274 [==============================] - 0s 91us/step - loss: 5.4015 - val_loss: 7.7706
Epoch 773/1000
274/274 [==============================] - 0s 95us/step - loss: 5.5151 - val_loss: 8.2058
Epoch 774/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3698 - val_loss: 7.9478
Epoch 775/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4656 - val_loss: 8.3621
Epoch 776/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3558 - val_loss: 7.8753
Epoch 777/1000
274/274 [==============================] - 0s 105us/step - loss: 5.4859 - val_loss: 8.2045
Epoch 778/1000
274/274 [==============================] - 0s 108us/step - loss: 5.3400 - val_loss: 7.9205
Epoch 779/1000
274/274 [==============================] - 0s 101us/s

Epoch 848/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4126 - val_loss: 7.7370
Epoch 849/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1603 - val_loss: 7.8576
Epoch 850/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1768 - val_loss: 7.6931
Epoch 851/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1300 - val_loss: 7.8535
Epoch 852/1000
274/274 [==============================] - 0s 91us/step - loss: 5.3310 - val_loss: 7.6948
Epoch 853/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1423 - val_loss: 7.6163
Epoch 854/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2879 - val_loss: 7.8747
Epoch 855/1000
274/274 [==============================] - 0s 111us/step - loss: 5.1721 - val_loss: 7.8008
Epoch 856/1000
274/274 [==============================] - 0s 114us/step - loss: 5.2038 - val_loss: 7.9163
Epoch 857/1000
274/274 [=============================

Epoch 926/1000
274/274 [==============================] - 0s 94us/step - loss: 5.1293 - val_loss: 7.8182
Epoch 927/1000
274/274 [==============================] - 0s 97us/step - loss: 5.1004 - val_loss: 7.9410
Epoch 928/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1917 - val_loss: 7.8724
Epoch 929/1000
274/274 [==============================] - 0s 93us/step - loss: 5.2548 - val_loss: 7.8174
Epoch 930/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2604 - val_loss: 7.8793
Epoch 931/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2275 - val_loss: 7.8540
Epoch 932/1000
274/274 [==============================] - 0s 99us/step - loss: 5.1773 - val_loss: 7.8519
Epoch 933/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2038 - val_loss: 7.8044
Epoch 934/1000
274/274 [==============================] - 0s 93us/step - loss: 5.0918 - val_loss: 7.7023
Epoch 935/1000
274/274 [==============================]

[array([[ 0.28635108,  2.9167414 , -0.23144896,  0.68618214, -0.7680612 ],
        [-0.48407128,  1.3466688 ,  1.0518687 ,  1.2875695 , -0.70855916],
        [-0.2260401 , -1.4053946 , -1.451052  ,  0.64102286, -0.48801693],
        [-0.1558429 ,  0.03466629, -2.008324  , -2.8242705 , -0.0749555 ],
        [-0.73916876,  0.55035764,  1.0508839 ,  0.8255887 , -0.39244083],
        [ 0.5261591 ,  0.96124864,  0.84337956,  1.5368651 , -0.1428345 ],
        [ 1.3142924 , -2.7074087 , -1.1096908 ,  0.4523525 ,  1.0735618 ]],
       dtype=float32),
 array([ 2.1329527,  1.9479085,  0.5942557, -2.1764657, -1.040667 ],
       dtype=float32),
 array([[ 0.6702776 ,  0.29393542,  0.5812508 ,  0.58492976,  0.41550812,
          0.09322894,  0.33460608,  0.72041297,  0.17871282, -0.73566467],
        [ 0.9246796 , -0.6927309 , -0.00647283,  1.0504009 ,  0.6562957 ,
          1.0918528 , -0.8806661 ,  0.3345802 ,  0.0279017 ,  0.10776122],
        [-0.4444281 , -0.21675685,  0.6993185 , -0.07312663, 

In [33]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_linear_2nd.h5')

In [34]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_relu(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_11 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_12 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 768us/step - loss:

Epoch 69/1000
274/274 [==============================] - 0s 102us/step - loss: 7.7365 - val_loss: 6.6486
Epoch 70/1000
274/274 [==============================] - 0s 113us/step - loss: 7.4913 - val_loss: 7.4083
Epoch 71/1000
274/274 [==============================] - 0s 102us/step - loss: 7.6619 - val_loss: 6.6389
Epoch 72/1000
274/274 [==============================] - 0s 94us/step - loss: 7.4948 - val_loss: 6.6729
Epoch 73/1000
274/274 [==============================] - 0s 98us/step - loss: 7.7573 - val_loss: 6.7043
Epoch 74/1000
274/274 [==============================] - 0s 98us/step - loss: 7.7216 - val_loss: 6.3907
Epoch 75/1000
274/274 [==============================] - 0s 98us/step - loss: 7.4887 - val_loss: 6.7755
Epoch 76/1000
274/274 [==============================] - 0s 103us/step - loss: 7.7938 - val_loss: 6.4753
Epoch 77/1000
274/274 [==============================] - 0s 98us/step - loss: 7.4877 - val_loss: 7.1544
Epoch 78/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 98us/step - loss: 7.1008 - val_loss: 6.5620
Epoch 147/1000
274/274 [==============================] - 0s 106us/step - loss: 7.0682 - val_loss: 6.6758
Epoch 148/1000
274/274 [==============================] - 0s 98us/step - loss: 7.2274 - val_loss: 6.5394
Epoch 149/1000
274/274 [==============================] - 0s 98us/step - loss: 7.1455 - val_loss: 6.4379
Epoch 150/1000
274/274 [==============================] - 0s 98us/step - loss: 7.1162 - val_loss: 6.4538
Epoch 151/1000
274/274 [==============================] - 0s 106us/step - loss: 7.0972 - val_loss: 6.7274
Epoch 152/1000
274/274 [==============================] - 0s 98us/step - loss: 7.0538 - val_loss: 6.5956
Epoch 153/1000
274/274 [==============================] - 0s 102us/step - loss: 6.9928 - val_loss: 6.6743
Epoch 154/1000
274/274 [==============================] - 0s 102us/step - loss: 7.1250 - val_loss: 6.7280
Epoch 155/1000
274/274 [==============================] - 0s 98us/

274/274 [==============================] - 0s 113us/step - loss: 6.8828 - val_loss: 6.6083
Epoch 224/1000
274/274 [==============================] - 0s 106us/step - loss: 6.9098 - val_loss: 6.7160
Epoch 225/1000
274/274 [==============================] - 0s 98us/step - loss: 7.4233 - val_loss: 6.7660
Epoch 226/1000
274/274 [==============================] - 0s 98us/step - loss: 6.9177 - val_loss: 6.8945
Epoch 227/1000
274/274 [==============================] - 0s 102us/step - loss: 6.7843 - val_loss: 6.4641
Epoch 228/1000
274/274 [==============================] - 0s 106us/step - loss: 6.9580 - val_loss: 6.4500
Epoch 229/1000
274/274 [==============================] - 0s 106us/step - loss: 6.9359 - val_loss: 6.7008
Epoch 230/1000
274/274 [==============================] - 0s 91us/step - loss: 6.9332 - val_loss: 6.3353
Epoch 231/1000
274/274 [==============================] - 0s 110us/step - loss: 6.8823 - val_loss: 6.5801
Epoch 232/1000
274/274 [==============================] - 0s 98u

274/274 [==============================] - 0s 107us/step - loss: 6.7227 - val_loss: 6.8042
Epoch 301/1000
274/274 [==============================] - 0s 109us/step - loss: 6.7466 - val_loss: 6.4613
Epoch 302/1000
274/274 [==============================] - 0s 113us/step - loss: 6.6997 - val_loss: 6.9247
Epoch 303/1000
274/274 [==============================] - 0s 113us/step - loss: 6.6940 - val_loss: 6.6712
Epoch 304/1000
274/274 [==============================] - 0s 106us/step - loss: 6.7496 - val_loss: 6.8849
Epoch 305/1000
274/274 [==============================] - 0s 101us/step - loss: 6.9357 - val_loss: 6.8440
Epoch 306/1000
274/274 [==============================] - 0s 103us/step - loss: 6.7971 - val_loss: 6.9837
Epoch 307/1000
274/274 [==============================] - 0s 101us/step - loss: 6.6484 - val_loss: 6.5724
Epoch 308/1000
274/274 [==============================] - 0s 98us/step - loss: 6.9988 - val_loss: 7.4846
Epoch 309/1000
274/274 [==============================] - 0s 9

...

274/274 [==============================] - 0s 102us/step - loss: 6.0497 - val_loss: 7.5863
Epoch 763/1000
274/274 [==============================] - 0s 102us/step - loss: 6.1002 - val_loss: 7.4730
Epoch 764/1000
274/274 [==============================] - 0s 103us/step - loss: 6.0324 - val_loss: 7.5288
Epoch 765/1000
274/274 [==============================] - 0s 98us/step - loss: 6.0286 - val_loss: 7.4694
Epoch 766/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0288 - val_loss: 7.3934
Epoch 767/1000
274/274 [==============================] - 0s 102us/step - loss: 6.1195 - val_loss: 7.2267
Epoch 768/1000
274/274 [==============================] - 0s 103us/step - loss: 5.9565 - val_loss: 7.5745
Epoch 769/1000
274/274 [==============================] - 0s 95us/step - loss: 5.9325 - val_loss: 7.8895
Epoch 770/1000
274/274 [==============================] - 0s 95us/step - loss: 6.1052 - val_loss: 7.3948
Epoch 771/1000
274/274 [==============================] - 0s 109u

274/274 [==============================] - 0s 109us/step - loss: 5.8685 - val_loss: 7.4473
Epoch 840/1000
274/274 [==============================] - ETA: 0s - loss: 2.149 - 0s 102us/step - loss: 5.9742 - val_loss: 7.3747
Epoch 841/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9502 - val_loss: 7.7020
Epoch 842/1000
274/274 [==============================] - 0s 100us/step - loss: 6.0007 - val_loss: 7.5988
Epoch 843/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9008 - val_loss: 7.3730
Epoch 844/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9552 - val_loss: 7.5037
Epoch 845/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8928 - val_loss: 7.6056
Epoch 846/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0571 - val_loss: 7.7234
Epoch 847/1000
274/274 [==============================] - 0s 99us/step - loss: 5.9282 - val_loss: 7.7128
Epoch 848/1000
274/274 [=================

274/274 [==============================] - 0s 98us/step - loss: 5.9041 - val_loss: 7.2781
Epoch 917/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9315 - val_loss: 7.7101
Epoch 918/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9063 - val_loss: 7.2375
Epoch 919/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8910 - val_loss: 7.6352
Epoch 920/1000
274/274 [==============================] - 0s 95us/step - loss: 6.2653 - val_loss: 7.7638
Epoch 921/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2364 - val_loss: 7.2948
Epoch 922/1000
274/274 [==============================] - 0s 91us/step - loss: 6.0248 - val_loss: 7.6227
Epoch 923/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2499 - val_loss: 8.1617
Epoch 924/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2264 - val_loss: 7.3078
Epoch 925/1000
274/274 [==============================] - 0s 98us/st

274/274 [==============================] - 0s 106us/step - loss: 6.1064 - val_loss: 7.6345
Epoch 994/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8710 - val_loss: 7.5262
Epoch 995/1000
274/274 [==============================] - 0s 95us/step - loss: 5.9020 - val_loss: 7.3175
Epoch 996/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8060 - val_loss: 7.7679
Epoch 997/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8790 - val_loss: 7.3873
Epoch 998/1000
274/274 [==============================] - 0s 95us/step - loss: 5.9302 - val_loss: 7.4819
Epoch 999/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8183 - val_loss: 7.3258
Epoch 1000/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9173 - val_loss: 7.8066
6.3480804087752


[array([[ 1.0224689 ,  0.21921225, -1.1375356 , -0.08617499, -0.6026804 ],
        [ 0.3352115 , -1.1281157 ,  2.3970206 , -1.3281008 ,  1.9482923 ],
        [-0.9176933 , -0.5032424 ,  0.69182867, -0.5090911 ,  0.6385779 ],
        [-1.8379833 ,  0.02913589, -0.9780376 ,  0.72872293, -0.9419593 ],
        [ 0.32177228, -1.1819646 , -0.4367179 ,  0.804513  , -0.80972   ],
        [ 0.5592985 , -1.0580313 ,  0.41903102,  1.0395093 , -0.6335145 ],
        [ 0.9933881 , -0.78222674, -0.23747157,  0.0714065 , -0.8467175 ]],
       dtype=float32),
 array([-0.02740788,  1.3033552 ,  0.9431078 ,  0.26118585, -0.67131895],
       dtype=float32),
 array([[-0.48873132,  0.6381111 ,  0.77224   , -0.48103845, -0.30789974,
          1.1727566 , -0.34275046, -0.6356171 ,  0.71848613,  0.36373696],
        [-0.45638183, -0.03181658,  0.5789742 , -0.17507464,  1.5215681 ,
          0.5056999 , -0.687918  , -0.34925434,  0.23410194,  0.49957067],
        [-0.5559578 ,  1.7359191 ,  0.96456647, -0.75845

In [35]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_relu_2nd.h5')

In [36]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_sigmoid(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_13 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_14 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 770us/step - loss:

Epoch 68/1000
274/274 [==============================] - 0s 109us/step - loss: 21.7279 - val_loss: 22.9875
Epoch 69/1000
274/274 [==============================] - 0s 94us/step - loss: 21.5534 - val_loss: 22.8507
Epoch 70/1000
274/274 [==============================] - 0s 109us/step - loss: 21.3175 - val_loss: 22.5180
Epoch 71/1000
274/274 [==============================] - 0s 107us/step - loss: 21.1537 - val_loss: 22.6243
Epoch 72/1000
274/274 [==============================] - 0s 98us/step - loss: 20.9100 - val_loss: 22.4975
Epoch 73/1000
274/274 [==============================] - 0s 102us/step - loss: 20.7677 - val_loss: 22.3134
Epoch 74/1000
274/274 [==============================] - 0s 102us/step - loss: 20.5264 - val_loss: 21.5115
Epoch 75/1000
274/274 [==============================] - 0s 97us/step - loss: 20.3503 - val_loss: 21.9178
Epoch 76/1000
274/274 [==============================] - 0s 101us/step - loss: 20.1753 - val_loss: 21.8753
Epoch 77/1000
274/274 [=================

274/274 [==============================] - 0s 101us/step - loss: 13.3392 - val_loss: 18.4650
Epoch 145/1000
274/274 [==============================] - 0s 106us/step - loss: 13.2081 - val_loss: 18.2924
Epoch 146/1000
274/274 [==============================] - 0s 98us/step - loss: 13.2246 - val_loss: 18.2873
Epoch 147/1000
274/274 [==============================] - 0s 102us/step - loss: 13.1411 - val_loss: 18.3032
Epoch 148/1000
274/274 [==============================] - 0s 102us/step - loss: 13.0662 - val_loss: 18.2494
Epoch 149/1000
274/274 [==============================] - 0s 102us/step - loss: 12.9177 - val_loss: 18.1826
Epoch 150/1000
274/274 [==============================] - 0s 103us/step - loss: 12.8575 - val_loss: 18.1444
Epoch 151/1000
274/274 [==============================] - 0s 95us/step - loss: 12.7901 - val_loss: 18.1119
Epoch 152/1000
274/274 [==============================] - 0s 98us/step - loss: 12.7404 - val_loss: 17.9444
Epoch 153/1000
274/274 [======================

274/274 [==============================] - 0s 102us/step - loss: 8.0348 - val_loss: 14.0778
Epoch 221/1000
274/274 [==============================] - 0s 98us/step - loss: 7.9910 - val_loss: 14.2091
Epoch 222/1000
274/274 [==============================] - 0s 109us/step - loss: 7.9160 - val_loss: 14.0255
Epoch 223/1000
274/274 [==============================] - 0s 102us/step - loss: 7.8879 - val_loss: 13.9312
Epoch 224/1000
274/274 [==============================] - 0s 95us/step - loss: 7.8750 - val_loss: 13.6548
Epoch 225/1000
274/274 [==============================] - 0s 95us/step - loss: 7.8395 - val_loss: 13.6553
Epoch 226/1000
274/274 [==============================] - 0s 102us/step - loss: 7.7877 - val_loss: 13.7591
Epoch 227/1000
274/274 [==============================] - 0s 102us/step - loss: 7.7336 - val_loss: 13.8079
Epoch 228/1000
274/274 [==============================] - 0s 98us/step - loss: 7.7555 - val_loss: 13.7552
Epoch 229/1000
274/274 [==============================] 

Epoch 297/1000
274/274 [==============================] - 0s 113us/step - loss: 6.5823 - val_loss: 11.8915
Epoch 298/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6155 - val_loss: 11.8732
Epoch 299/1000
274/274 [==============================] - 0s 124us/step - loss: 6.5724 - val_loss: 11.8485
Epoch 300/1000
274/274 [==============================] - 0s 153us/step - loss: 6.5599 - val_loss: 11.8918
Epoch 301/1000
274/274 [==============================] - 0s 106us/step - loss: 6.5319 - val_loss: 11.9125
Epoch 302/1000
274/274 [==============================] - 0s 87us/step - loss: 6.5696 - val_loss: 11.8389
Epoch 303/1000
274/274 [==============================] - 0s 91us/step - loss: 6.5617 - val_loss: 11.8989
Epoch 304/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6133 - val_loss: 11.9403
Epoch 305/1000
274/274 [==============================] - 0s 96us/step - loss: 6.4960 - val_loss: 11.8559
Epoch 306/1000
274/274 [=================

...

Epoch 758/1000
274/274 [==============================] - 0s 98us/step - loss: 4.9723 - val_loss: 10.1226
Epoch 759/1000
274/274 [==============================] - 0s 94us/step - loss: 4.9719 - val_loss: 9.9865
Epoch 760/1000
274/274 [==============================] - 0s 102us/step - loss: 5.0449 - val_loss: 10.2346
Epoch 761/1000
274/274 [==============================] - 0s 100us/step - loss: 4.9773 - val_loss: 10.1215
Epoch 762/1000
274/274 [==============================] - 0s 102us/step - loss: 4.9780 - val_loss: 9.9774
Epoch 763/1000
274/274 [==============================] - 0s 101us/step - loss: 5.0473 - val_loss: 9.9191
Epoch 764/1000
274/274 [==============================] - 0s 101us/step - loss: 4.9489 - val_loss: 10.0805
Epoch 765/1000
274/274 [==============================] - 0s 113us/step - loss: 4.9443 - val_loss: 9.9973
Epoch 766/1000
274/274 [==============================] - 0s 123us/step - loss: 4.9614 - val_loss: 10.0660
Epoch 767/1000
274/274 [===================

Epoch 835/1000
274/274 [==============================] - 0s 102us/step - loss: 4.8021 - val_loss: 10.0976
Epoch 836/1000
274/274 [==============================] - 0s 98us/step - loss: 4.7578 - val_loss: 10.2218
Epoch 837/1000
274/274 [==============================] - 0s 98us/step - loss: 4.7630 - val_loss: 10.2299
Epoch 838/1000
274/274 [==============================] - 0s 98us/step - loss: 4.8037 - val_loss: 10.1385
Epoch 839/1000
274/274 [==============================] - 0s 98us/step - loss: 4.7078 - val_loss: 10.0433
Epoch 840/1000
274/274 [==============================] - 0s 103us/step - loss: 4.7415 - val_loss: 10.1842
Epoch 841/1000
274/274 [==============================] - 0s 98us/step - loss: 4.7654 - val_loss: 10.0458
Epoch 842/1000
274/274 [==============================] - 0s 98us/step - loss: 4.8175 - val_loss: 10.1199
Epoch 843/1000
274/274 [==============================] - 0s 109us/step - loss: 4.8285 - val_loss: 9.9976
Epoch 844/1000
274/274 [====================

274/274 [==============================] - 0s 106us/step - loss: 4.6276 - val_loss: 9.9474
Epoch 913/1000
274/274 [==============================] - 0s 98us/step - loss: 4.5963 - val_loss: 9.8555
Epoch 914/1000
274/274 [==============================] - 0s 95us/step - loss: 4.5889 - val_loss: 9.8870
Epoch 915/1000
274/274 [==============================] - 0s 99us/step - loss: 4.6430 - val_loss: 9.9185
Epoch 916/1000
274/274 [==============================] - 0s 106us/step - loss: 4.5908 - val_loss: 9.9453
Epoch 917/1000
274/274 [==============================] - 0s 102us/step - loss: 4.5694 - val_loss: 9.8150
Epoch 918/1000
274/274 [==============================] - 0s 99us/step - loss: 4.6065 - val_loss: 9.9224
Epoch 919/1000
274/274 [==============================] - 0s 98us/step - loss: 4.5622 - val_loss: 9.8826
Epoch 920/1000
274/274 [==============================] - 0s 102us/step - loss: 4.5664 - val_loss: 9.9487
Epoch 921/1000
274/274 [==============================] - 0s 98us/

274/274 [==============================] - 0s 100us/step - loss: 4.4600 - val_loss: 9.5651
Epoch 990/1000
274/274 [==============================] - 0s 95us/step - loss: 4.4822 - val_loss: 9.5816
Epoch 991/1000
274/274 [==============================] - 0s 109us/step - loss: 4.4847 - val_loss: 9.3934
Epoch 992/1000
274/274 [==============================] - 0s 102us/step - loss: 4.5691 - val_loss: 9.3774
Epoch 993/1000
274/274 [==============================] - 0s 98us/step - loss: 4.5689 - val_loss: 9.6751
Epoch 994/1000
274/274 [==============================] - 0s 102us/step - loss: 4.5672 - val_loss: 9.6429
Epoch 995/1000
274/274 [==============================] - 0s 95us/step - loss: 4.6304 - val_loss: 9.3753
Epoch 996/1000
274/274 [==============================] - 0s 96us/step - loss: 4.4677 - val_loss: 9.4349
Epoch 997/1000
274/274 [==============================] - 0s 114us/step - loss: 4.5014 - val_loss: 9.6188
Epoch 998/1000
274/274 [==============================] - 0s 106u

[array([[ 2.3766391 ,  0.27925315,  1.4172825 ,  0.08412278,  1.7259717 ],
        [ 0.35011595,  1.0511056 , -0.45420435,  2.4204068 ,  0.35269186],
        [-1.292106  , -2.0360212 ,  2.5093586 , -0.37578198, -3.6503513 ],
        [ 0.576231  , -2.074081  ,  2.023471  , -2.3220212 , -1.7237614 ],
        [ 1.3628268 , -0.5754887 , -0.16974244, -0.14067908,  0.2831491 ],
        [-1.7802343 ,  2.1303613 , -1.1232035 ,  2.1423337 ,  0.66110224],
        [-2.8325272 ,  3.0432506 ,  0.99948084,  1.1257128 ,  0.76806134]],
       dtype=float32),
 array([ 0.78084594,  1.6883808 , -0.38531366, -0.63162446, -0.39923006],
       dtype=float32),
 array([[ 4.2405343 ,  3.8364203 ,  1.5094736 ,  2.054304  , -0.89266986,
          4.8991675 , -1.8251492 , -0.06398264,  0.14101045, -0.9005411 ],
        [ 4.010821  ,  2.8878114 , -0.6034095 ,  2.3255572 , -0.17043495,
          2.8649137 , -1.8925574 ,  9.337868  , -5.7227287 ,  0.5320479 ],
        [-0.38833794, -0.39229506,  2.071479  , -0.31024

In [37]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_sigmoid_2nd.h5')

In [38]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_tanh(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_15 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_16 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 823us/step - loss:

274/274 [==============================] - 0s 98us/step - loss: 61.4260 - val_loss: 60.1099
Epoch 68/1000
274/274 [==============================] - 0s 102us/step - loss: 61.3926 - val_loss: 60.1102
Epoch 69/1000
274/274 [==============================] - 0s 102us/step - loss: 61.3957 - val_loss: 60.1893
Epoch 70/1000
274/274 [==============================] - 0s 98us/step - loss: 61.3843 - val_loss: 60.2175
Epoch 71/1000
274/274 [==============================] - 0s 116us/step - loss: 61.3777 - val_loss: 60.1537
Epoch 72/1000
274/274 [==============================] - 0s 116us/step - loss: 61.3769 - val_loss: 60.1321
Epoch 73/1000
274/274 [==============================] - 0s 116us/step - loss: 61.3975 - val_loss: 60.1799
Epoch 74/1000
274/274 [==============================] - 0s 120us/step - loss: 61.3646 - val_loss: 60.0070
Epoch 75/1000
274/274 [==============================] - 0s 106us/step - loss: 61.3709 - val_loss: 59.9346
Epoch 76/1000
274/274 [==============================

274/274 [==============================] - 0s 109us/step - loss: 11.4867 - val_loss: 15.1101
Epoch 144/1000
274/274 [==============================] - 0s 113us/step - loss: 11.4160 - val_loss: 14.9813
Epoch 145/1000
274/274 [==============================] - 0s 98us/step - loss: 11.3147 - val_loss: 14.9335
Epoch 146/1000
274/274 [==============================] - 0s 102us/step - loss: 11.2269 - val_loss: 14.7532
Epoch 147/1000
274/274 [==============================] - 0s 102us/step - loss: 9.9162 - val_loss: 13.5156
Epoch 148/1000
274/274 [==============================] - 0s 102us/step - loss: 9.7289 - val_loss: 13.2051
Epoch 149/1000
274/274 [==============================] - 0s 98us/step - loss: 9.4134 - val_loss: 12.6610
Epoch 150/1000
274/274 [==============================] - 0s 106us/step - loss: 8.9483 - val_loss: 12.1228
Epoch 151/1000
274/274 [==============================] - 0s 109us/step - loss: 8.7279 - val_loss: 11.7241
Epoch 152/1000
274/274 [==========================

Epoch 220/1000
274/274 [==============================] - 0s 113us/step - loss: 6.7187 - val_loss: 9.7736
Epoch 221/1000
274/274 [==============================] - 0s 109us/step - loss: 6.7868 - val_loss: 10.0621
Epoch 222/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6466 - val_loss: 9.9954
Epoch 223/1000
274/274 [==============================] - 0s 97us/step - loss: 6.7309 - val_loss: 10.1814
Epoch 224/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6392 - val_loss: 9.7424
Epoch 225/1000
274/274 [==============================] - 0s 105us/step - loss: 6.8102 - val_loss: 10.0544
Epoch 226/1000
274/274 [==============================] - 0s 95us/step - loss: 6.6000 - val_loss: 9.9357
Epoch 227/1000
274/274 [==============================] - 0s 106us/step - loss: 6.5665 - val_loss: 9.6785
Epoch 228/1000
274/274 [==============================] - 0s 102us/step - loss: 6.5417 - val_loss: 9.8043
Epoch 229/1000
274/274 [======================

274/274 [==============================] - 0s 102us/step - loss: 6.9852 - val_loss: 10.0793
Epoch 298/1000
274/274 [==============================] - 0s 109us/step - loss: 6.5919 - val_loss: 9.7021
Epoch 299/1000
274/274 [==============================] - 0s 95us/step - loss: 6.2712 - val_loss: 9.5127
Epoch 300/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4867 - val_loss: 9.7457
Epoch 301/1000
274/274 [==============================] - 0s 101us/step - loss: 6.2731 - val_loss: 9.9009
Epoch 302/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2152 - val_loss: 9.8249
Epoch 303/1000
274/274 [==============================] - 0s 104us/step - loss: 6.2036 - val_loss: 9.7317
Epoch 304/1000
274/274 [==============================] - 0s 106us/step - loss: 6.2456 - val_loss: 9.6846
Epoch 305/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2014 - val_loss: 9.6799
Epoch 306/1000
274/274 [==============================] - 0s 9

...

274/274 [==============================] - 0s 98us/step - loss: 5.5700 - val_loss: 10.0090
Epoch 760/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2905 - val_loss: 10.2476
Epoch 761/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3717 - val_loss: 10.0448
Epoch 762/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2732 - val_loss: 10.2928
Epoch 763/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2635 - val_loss: 9.9847
Epoch 764/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3693 - val_loss: 10.0019
Epoch 765/1000
274/274 [==============================] - 0s 105us/step - loss: 5.2847 - val_loss: 10.0815
Epoch 766/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4943 - val_loss: 10.1561
Epoch 767/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4671 - val_loss: 10.2197
Epoch 768/1000
274/274 [==============================] - 

Epoch 836/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2153 - val_loss: 10.4402
Epoch 837/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3918 - val_loss: 10.3119
Epoch 838/1000
274/274 [==============================] - 0s 106us/step - loss: 5.6312 - val_loss: 10.6877
Epoch 839/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4445 - val_loss: 10.6955
Epoch 840/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6850 - val_loss: 10.5864
Epoch 841/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7964 - val_loss: 10.5711
Epoch 842/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3956 - val_loss: 10.4749
Epoch 843/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3548 - val_loss: 10.5585
Epoch 844/1000
274/274 [==============================] - 0s 91us/step - loss: 5.5257 - val_loss: 10.6387
Epoch 845/1000
274/274 [===================

Epoch 913/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4219 - val_loss: 10.8089
Epoch 914/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3485 - val_loss: 10.3182
Epoch 915/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1990 - val_loss: 11.0065
Epoch 916/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5336 - val_loss: 10.0495
Epoch 917/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2806 - val_loss: 10.1265
Epoch 918/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1393 - val_loss: 10.1836
Epoch 919/1000
274/274 [==============================] - 0s 106us/step - loss: 5.0752 - val_loss: 10.4582
Epoch 920/1000
274/274 [==============================] - 0s 106us/step - loss: 5.2376 - val_loss: 10.2810
Epoch 921/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4179 - val_loss: 10.3106
Epoch 922/1000
274/274 [=================

274/274 [==============================] - 0s 102us/step - loss: 5.1515 - val_loss: 9.9660
Epoch 991/1000
274/274 [==============================] - 0s 106us/step - loss: 5.0848 - val_loss: 9.9932
Epoch 992/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1268 - val_loss: 9.8425
Epoch 993/1000
274/274 [==============================] - 0s 98us/step - loss: 5.0788 - val_loss: 9.9182
Epoch 994/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1493 - val_loss: 10.1000
Epoch 995/1000
274/274 [==============================] - 0s 102us/step - loss: 5.0334 - val_loss: 9.8916
Epoch 996/1000
274/274 [==============================] - 0s 104us/step - loss: 5.1068 - val_loss: 9.9012
Epoch 997/1000
274/274 [==============================] - 0s 106us/step - loss: 5.1804 - val_loss: 10.0026
Epoch 998/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3021 - val_loss: 10.0681
Epoch 999/1000
274/274 [==============================] - 0s 

[array([[-0.93721586,  1.0142405 , -0.63533235,  1.4860946 ,  1.6661663 ],
        [-1.2892848 ,  0.13725366, -1.0635946 , -1.0897348 ,  0.2607438 ],
        [ 0.9041363 , -2.4055266 , -0.1755637 ,  1.5018466 , -2.0919046 ],
        [ 0.6482353 , -0.88861954, -0.7893984 ,  1.5678409 , -0.15092516],
        [-0.3523444 , -0.5832104 , -0.25203574,  0.50616693, -0.77049595],
        [ 0.6109077 ,  1.5072347 ,  0.62969667, -1.0125368 ,  0.16259655],
        [-0.06370833,  0.5220317 , -0.9184296 , -1.7730603 ,  0.47551584]],
       dtype=float32),
 array([ 0.9841386 , -0.71666616,  0.35762247,  0.56887317, -0.3944741 ],
       dtype=float32),
 array([[ -0.4670606 ,  -2.1155608 ,  -0.23940441,   0.5950548 ,
           1.6993604 ,  -2.3058186 ,  -2.748654  ,   0.47687435,
          -9.090973  ,   0.78415   ],
        [ -1.4978926 ,  -0.98516583,   0.6066328 ,   1.4342366 ,
           4.276979  ,  -0.4254415 ,  -1.170297  ,   0.15900937,
         -10.3958845 ,   0.32857457],
        [ -4.95431

In [39]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_tanh_2nd.h5')

In [40]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_linear(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_17 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_26 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_18 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 0

143/143 [==============================] - 0s 119us/step - loss: 0.0064 - val_loss: 0.0053
Epoch 70/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0057 - val_loss: 0.0056
Epoch 71/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0055 - val_loss: 0.0053
Epoch 72/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0058 - val_loss: 0.0056
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0057 - val_loss: 0.0051
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0057 - val_loss: 0.0051
Epoch 75/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0059 - val_loss: 0.0055
Epoch 76/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0061 - val_loss: 0.0049
Epoch 77/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0058 - val_loss: 0.0051
Epoch 78/1000
143/143 [==============================] - 0s 105us/ste

143/143 [==============================] - 0s 112us/step - loss: 0.0045 - val_loss: 0.0046
Epoch 147/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0045
Epoch 148/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0049
Epoch 149/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0044 - val_loss: 0.0048
Epoch 150/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0047
Epoch 151/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0046
Epoch 152/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0044 - val_loss: 0.0048
Epoch 153/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0048
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0046 - val_loss: 0.0045
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 118us/step - loss: 0.0035 - val_loss: 0.0039
Epoch 224/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0036 - val_loss: 0.0047
Epoch 225/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0035 - val_loss: 0.0043
Epoch 226/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0035 - val_loss: 0.0050
Epoch 227/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0039 - val_loss: 0.0044
Epoch 228/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0044 - val_loss: 0.0053
Epoch 229/1000
143/143 [==============================] - 0s 160us/step - loss: 0.0039 - val_loss: 0.0050
Epoch 230/1000
143/143 [==============================] - 0s 153us/step - loss: 0.0038 - val_loss: 0.0045
Epoch 231/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0037 - val_loss: 0.0051
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0030 - val_loss: 0.0040
Epoch 301/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0030 - val_loss: 0.0040
Epoch 302/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0030 - val_loss: 0.0055
Epoch 303/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0030 - val_loss: 0.0057
Epoch 304/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0034 - val_loss: 0.0076
Epoch 305/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0044 - val_loss: 0.0045
Epoch 306/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0036 - val_loss: 0.0045
Epoch 307/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0046 - val_loss: 0.0062
Epoch 308/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0042 - val_loss: 0.0057
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0045
Epoch 763/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0020 - val_loss: 0.0043
Epoch 764/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0015 - val_loss: 0.0051
Epoch 765/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0041
Epoch 766/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0043
Epoch 767/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0014 - val_loss: 0.0053
Epoch 768/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0041
Epoch 769/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0040
Epoch 770/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0019 - val_loss: 0.0042
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0046
Epoch 840/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0041
Epoch 841/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0042
Epoch 842/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0041
Epoch 843/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0013 - val_loss: 0.0048
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0045
Epoch 845/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0013 - val_loss: 0.0042
Epoch 846/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0013 - val_loss: 0.0053
Epoch 847/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0013 - val_loss: 0.0040
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0045
Epoch 917/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0015 - val_loss: 0.0042
Epoch 918/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0049
Epoch 919/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0012 - val_loss: 0.0048
Epoch 920/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0013 - val_loss: 0.0040
Epoch 921/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0018 - val_loss: 0.0061
Epoch 922/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0016 - val_loss: 0.0046
Epoch 923/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0014 - val_loss: 0.0043
Epoch 924/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0013 - val_loss: 0.0058
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0012 - val_loss: 0.0051
Epoch 994/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0055
Epoch 995/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0013 - val_loss: 0.0047
Epoch 996/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0014 - val_loss: 0.0045
Epoch 997/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0012 - val_loss: 0.0047
Epoch 998/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0056
Epoch 999/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0045
Epoch 1000/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0013 - val_loss: 0.0045
0.005055123008787632


[array([[ 2.07926527e-01, -2.45262131e-01, -7.91803420e-01,
          2.30019301e-01, -1.86502531e-01],
        [ 2.80307643e-02, -1.10159302e+00,  2.63107359e-01,
         -9.17307362e-02, -8.44111383e-01],
        [-1.01208949e+00, -5.04398048e-01,  2.64170080e-01,
          8.46432447e-02, -1.40624210e-01],
        [-8.03521395e-01,  5.74957013e-01, -3.62008542e-01,
          1.89931616e-01,  6.89588130e-01],
        [ 1.43744722e-01,  4.79616553e-01, -5.99399395e-02,
         -4.43951964e-01, -3.17525715e-01],
        [ 1.37709546e+00,  4.73687351e-02, -2.30803639e-01,
         -3.63167882e-01, -1.36871701e-02],
        [-6.27183855e-01, -4.81542051e-01,  2.15626419e-01,
          4.85081449e-02,  7.37508953e-01],
        [ 2.27012619e-01,  4.11708169e-02, -1.06609434e-01,
          1.23651810e-01, -4.22493994e-01],
        [ 3.86635900e-01,  1.95587707e+00,  6.64027035e-01,
         -7.17437267e-01,  4.43882048e-01],
        [-5.06133884e-02, -1.24602056e+00, -3.52523953e-01,
    

In [41]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_linear_2nd.h5')

In [42]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_relu(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_19 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_20 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 105us/step - loss: 0.0370 - val_loss: 0.0282
Epoch 70/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 71/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0260
Epoch 72/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0260
Epoch 73/1000
143/143 [==============================] - 0s 124us/step - loss: 0.0369 - val_loss: 0.0266
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0272
Epoch 75/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0274
Epoch 76/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0368 - val_loss: 0.0269
Epoch 77/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0264
Epoch 78/1000
143/143 [==============================] - 0s 105us/ste

143/143 [==============================] - 0s 114us/step - loss: 0.0368 - val_loss: 0.0271
Epoch 147/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0368 - val_loss: 0.0272
Epoch 148/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 149/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0269
Epoch 150/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 151/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0272
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0271
Epoch 153/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0265
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0273
Epoch 224/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0265
Epoch 225/1000
143/143 [==============================] - 0s 124us/step - loss: 0.0369 - val_loss: 0.0262
Epoch 226/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0369 - val_loss: 0.0265
Epoch 227/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 228/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0272
Epoch 229/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0274
Epoch 230/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0273
Epoch 231/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0278
Epoch 301/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0274
Epoch 302/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 303/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0267
Epoch 304/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0370 - val_loss: 0.0264
Epoch 305/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 306/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0368 - val_loss: 0.0279
Epoch 307/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0371 - val_loss: 0.0284
Epoch 308/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0273
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0274
Epoch 763/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0274
Epoch 764/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0368 - val_loss: 0.0271
Epoch 765/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0264
Epoch 766/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0368 - val_loss: 0.0265
Epoch 767/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 768/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0274
Epoch 769/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0368 - val_loss: 0.0274
Epoch 770/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0269
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0272
Epoch 840/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0370 - val_loss: 0.0261
Epoch 841/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0262
Epoch 842/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 843/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0369 - val_loss: 0.0277
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0278
Epoch 845/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0272
Epoch 846/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 847/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0280
Epoch 917/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0276
Epoch 918/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 919/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0263
Epoch 920/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0260
Epoch 921/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0266
Epoch 922/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 923/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0368 - val_loss: 0.0275
Epoch 924/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0368 - val_loss: 0.0279
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 120us/step - loss: 0.0370 - val_loss: 0.0260
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0271
Epoch 995/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0369 - val_loss: 0.0280
Epoch 996/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0276
Epoch 997/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0269
Epoch 998/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 999/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 1000/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0369 - val_loss: 0.0273
0.05467016622424126


[array([[-0.38070178,  0.05093754,  0.28398916,  0.04300103, -0.68727183],
        [-0.1014373 , -0.5660536 , -0.28945524,  0.18106064, -0.3436229 ],
        [-0.82323325, -0.0955064 , -0.0691985 , -0.08945966, -0.2065902 ],
        [-0.3538584 , -0.6491721 , -0.78472453, -0.45083833, -0.1546531 ],
        [ 0.05209605, -0.19480152, -0.41885355, -0.22170794, -0.69067883],
        [-0.5076685 , -0.1789403 , -0.25681943,  0.31936643,  0.05067737],
        [-0.6441425 , -0.6381071 , -0.5410501 , -0.41913873, -0.7073273 ],
        [-0.5863256 , -0.02686948, -0.15920386, -0.25277442, -0.7365268 ],
        [-0.5942957 , -0.18908627, -0.2615877 , -0.2654875 , -0.02388259],
        [-0.03458009, -0.68694305, -0.54144293, -0.4332741 , -0.1418482 ],
        [-0.32284656,  0.04954637, -0.61614275,  0.45769444, -0.8198087 ],
        [-0.5180238 , -0.3845863 , -0.40332273,  0.20796862,  0.04363811],
        [-0.11034798, -0.57044834,  0.12929687, -0.33118182,  0.01734884],
        [-0.3229761 , -0.

In [43]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_relu_2nd.h5')

In [44]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_sigmoid(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_21 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 133us/step - loss: 0.0106 - val_loss: 0.0064
Epoch 70/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0111 - val_loss: 0.0054
Epoch 71/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0104 - val_loss: 0.0055
Epoch 72/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0105 - val_loss: 0.0057
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0114 - val_loss: 0.0055
Epoch 74/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0103 - val_loss: 0.0054
Epoch 75/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0103 - val_loss: 0.0059
Epoch 76/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0105 - val_loss: 0.0052
Epoch 77/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0101 - val_loss: 0.0052
Epoch 78/1000
143/143 [==============================] - 0s 119us/ste

143/143 [==============================] - 0s 119us/step - loss: 0.0075 - val_loss: 0.0051
Epoch 147/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0076 - val_loss: 0.0051
Epoch 148/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0078 - val_loss: 0.0050
Epoch 149/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0075 - val_loss: 0.0057
Epoch 150/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0075 - val_loss: 0.0050
Epoch 151/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0075 - val_loss: 0.0053
Epoch 152/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0075 - val_loss: 0.0053
Epoch 153/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0074 - val_loss: 0.0051
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0075 - val_loss: 0.0052
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0066 - val_loss: 0.0050
Epoch 224/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0071 - val_loss: 0.0050
Epoch 225/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0068 - val_loss: 0.0050
Epoch 226/1000
143/143 [==============================] - 0s 127us/step - loss: 0.0068 - val_loss: 0.0051
Epoch 227/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0067 - val_loss: 0.0048
Epoch 228/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0068 - val_loss: 0.0052
Epoch 229/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0068 - val_loss: 0.0049
Epoch 230/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0067 - val_loss: 0.0050
Epoch 231/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0067 - val_loss: 0.0050
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0062 - val_loss: 0.0048
Epoch 301/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0064 - val_loss: 0.0046
Epoch 302/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0062 - val_loss: 0.0052
Epoch 303/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0061 - val_loss: 0.0047
Epoch 304/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0062 - val_loss: 0.0052
Epoch 305/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0061 - val_loss: 0.0047
Epoch 306/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0060 - val_loss: 0.0048
Epoch 307/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0062 - val_loss: 0.0050
Epoch 308/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0065 - val_loss: 0.0045
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 112us/step - loss: 0.0045 - val_loss: 0.0050
Epoch 763/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0046 - val_loss: 0.0049
Epoch 764/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0046 - val_loss: 0.0047
Epoch 765/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0046 - val_loss: 0.0053
Epoch 766/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0047 - val_loss: 0.0046
Epoch 767/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0046 - val_loss: 0.0054
Epoch 768/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0048 - val_loss: 0.0046
Epoch 769/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0045 - val_loss: 0.0057
Epoch 770/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0049 - val_loss: 0.0046
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 118us/step - loss: 0.0045 - val_loss: 0.0047
Epoch 840/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0044 - val_loss: 0.0048
Epoch 841/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0044 - val_loss: 0.0049
Epoch 842/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0045 - val_loss: 0.0046
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0045
Epoch 844/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0045 - val_loss: 0.0049
Epoch 845/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0044 - val_loss: 0.0048
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0049
Epoch 847/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0045 - val_loss: 0.0050
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0049 - val_loss: 0.0047
Epoch 917/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0048 - val_loss: 0.0048
Epoch 918/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0046 - val_loss: 0.0046
Epoch 919/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0045 - val_loss: 0.0046
Epoch 920/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0045 - val_loss: 0.0048
Epoch 921/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0046 - val_loss: 0.0046
Epoch 922/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0045 - val_loss: 0.0048
Epoch 923/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0043 - val_loss: 0.0045
Epoch 924/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0043 - val_loss: 0.0051
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0050 - val_loss: 0.0050
Epoch 994/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0046 - val_loss: 0.0049
Epoch 995/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0046 - val_loss: 0.0046
Epoch 996/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0051
Epoch 997/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0048 - val_loss: 0.0045
Epoch 998/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0050 - val_loss: 0.0049
Epoch 999/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0046 - val_loss: 0.0045
Epoch 1000/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0047 - val_loss: 0.0048
0.008190426975488663


[array([[ 6.8932390e-01, -3.4172720e-01, -4.6320570e-01, -5.2318412e-01,
         -1.8387462e-01],
        [-1.3491477e-01, -2.2089702e-01, -1.3383634e-01, -6.9597280e-01,
         -5.5431855e-01],
        [-2.0201902e-01, -8.3530098e-02,  5.7456926e-02,  7.8027107e-02,
         -3.3995131e-01],
        [ 1.8516433e-01, -5.0327271e-02, -6.3716292e-01,  1.5194365e-01,
         -1.4779061e-01],
        [-2.1351235e-01, -1.8119872e-02,  7.3341899e-02, -5.9947900e-02,
         -6.1143851e-01],
        [ 9.1252488e-01, -1.8023443e-01, -8.0963099e-01,  1.3943671e-01,
         -4.0973967e-01],
        [-3.7120032e-01, -2.2296268e-01, -8.2238950e-02, -1.4245750e-01,
         -5.7540447e-01],
        [ 2.5287932e-01, -2.6341185e-01, -2.0357479e-01, -3.3594728e-01,
         -3.3455682e-01],
        [-7.3779918e-02, -3.5907689e-01, -7.3205662e-01, -6.0543364e-01,
         -1.7458627e-01],
        [ 9.0528846e-01,  1.9201013e-01, -8.0017632e-01, -1.0413492e-02,
         -8.8062353e-02],
        [-

In [45]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_sigmoid_2nd.h5')

In [46]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_tanh(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_23 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_35 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_24 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 112us/step - loss: 0.0371 - val_loss: 0.0262
Epoch 70/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0369 - val_loss: 0.0270
Epoch 71/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0277
Epoch 72/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0272
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0269
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0273
Epoch 75/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 76/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 77/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0370 - val_loss: 0.0267
Epoch 78/1000
143/143 [==============================] - 0s 112us/ste

143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0276
Epoch 147/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0370 - val_loss: 0.0282
Epoch 148/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0370 - val_loss: 0.0280
Epoch 149/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0370 - val_loss: 0.0271
Epoch 150/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0269
Epoch 151/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0369 - val_loss: 0.0278
Epoch 152/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0274
Epoch 153/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0265
Epoch 154/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 111us/step - loss: 0.0370 - val_loss: 0.0281
Epoch 224/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0369 - val_loss: 0.0275
Epoch 225/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0271
Epoch 226/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0262
Epoch 227/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0261
Epoch 228/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0369 - val_loss: 0.0265
Epoch 229/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0278
Epoch 230/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0284
Epoch 231/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0369 - val_loss: 0.0272
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0273
Epoch 301/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 302/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0262
Epoch 303/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0369 - val_loss: 0.0261
Epoch 304/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0265
Epoch 305/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0269
Epoch 306/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0369 - val_loss: 0.0275
Epoch 307/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0275
Epoch 308/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0273
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0277
Epoch 763/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0279
Epoch 764/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0278
Epoch 765/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0370 - val_loss: 0.0282
Epoch 766/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0369 - val_loss: 0.0273
Epoch 767/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0273
Epoch 768/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 769/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0369 - val_loss: 0.0264
Epoch 770/1000
143/143 [==============================] - 0s 153us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 771/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 126us/step - loss: 0.0369 - val_loss: 0.0273
Epoch 840/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0269
Epoch 841/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0273
Epoch 842/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0270
Epoch 843/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0369 - val_loss: 0.0273
Epoch 844/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 845/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 846/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0368 - val_loss: 0.0266
Epoch 847/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0369 - val_loss: 0.0265
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0370 - val_loss: 0.0271
Epoch 917/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 918/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0265
Epoch 919/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0265
Epoch 920/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0264
Epoch 921/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0271
Epoch 922/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0278
Epoch 923/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0287
Epoch 924/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0370 - val_loss: 0.0279
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 120us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 994/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0368 - val_loss: 0.0265
Epoch 995/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0368 - val_loss: 0.0265
Epoch 996/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0369 - val_loss: 0.0264
Epoch 997/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0370 - val_loss: 0.0268
Epoch 998/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0266
Epoch 999/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0368 - val_loss: 0.0269
Epoch 1000/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0269
0.054700180888175964


[array([[-0.70168304,  0.04651755,  0.10688117, -0.6131091 , -0.15943389],
        [-0.32554382, -0.6729048 , -0.14768043, -0.01486701,  0.0990205 ],
        [-0.03181594, -0.40534827, -0.40209556, -0.23426937, -0.22163515],
        [-0.36851957, -0.19336285,  0.39547738, -0.6070916 ,  0.18342051],
        [-0.357041  , -0.56808686, -0.29190534, -0.36353073, -0.3003848 ],
        [-0.58956254, -0.26148033, -0.29410654, -0.36828968, -0.6109175 ],
        [-0.72772324,  0.10183853, -0.3157071 , -0.380015  , -0.6470737 ],
        [-0.7074462 ,  0.06710318, -0.31836987, -0.14888819, -0.5229826 ],
        [-0.3365185 ,  0.21725246, -0.06206229, -0.38109732,  0.09730373],
        [ 0.01747735, -0.19986258, -0.27517205,  0.06829961, -0.7212097 ],
        [-0.21659635, -0.3896386 ,  0.1886845 , -0.22950661, -0.18680635],
        [-0.52136123, -0.01033403,  0.44676337, -0.73050267, -0.63059574],
        [-0.01871279,  0.19597559, -0.46867216, -0.63834685, -0.24275887],
        [-0.1381751 , -0.

In [47]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_tanh_2nd.h5')

In [48]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_linear(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_25 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_38 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_26 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 9ms/step - loss: 37.022

Epoch 71/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0138 - val_loss: 0.0174
Epoch 72/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0121 - val_loss: 0.0136
Epoch 73/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0113 - val_loss: 0.0112
Epoch 74/1000
32/32 [==============================] - 0s 154us/step - loss: 0.0112 - val_loss: 0.0104
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0116 - val_loss: 0.0107
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0122 - val_loss: 0.0117
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0126 - val_loss: 0.0128
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0129 - val_loss: 0.0136
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0127 - val_loss: 0.0136
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0039 - val_loss: 0.0047
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 0.0047
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 0.0046
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0037 - val_loss: 0.0046
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0037 - val_loss: 0.0046
Epoch 156/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0037 - val_loss: 0.0045
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0036 - val_loss: 0.0045
Epoch 158/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0036 - val_loss: 0.0044
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0036 - val_loss: 0.0044
Epoch 160/1000
32/32 [==============================] - 0s 155us

Epoch 230/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0019 - val_loss: 0.0019
Epoch 231/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0019 - val_loss: 0.0019
Epoch 232/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0019 - val_loss: 0.0019
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0019 - val_loss: 0.0019
Epoch 234/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0019 - val_loss: 0.0019
Epoch 235/1000
32/32 [==============================] - 0s 188us/step - loss: 0.0018 - val_loss: 0.0018
Epoch 236/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0018 - val_loss: 0.0018
Epoch 237/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0018
Epoch 238/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0018
Epoch 239/1000
32/32 [==============================] - 0s 155us

Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0012 - val_loss: 8.9693e-04
Epoch 310/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0011 - val_loss: 8.8898e-04
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 8.8115e-04
Epoch 312/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0011 - val_loss: 8.7339e-04
Epoch 313/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 8.6573e-04
Epoch 314/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 8.5815e-04
Epoch 315/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 8.5067e-04
Epoch 316/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 8.4326e-04
Epoch 317/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0011 - val_loss: 8.3593e-04
Epoch 318/1000
32/32 [======

...

Epoch 754/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5096e-04 - val_loss: 1.3290e-04
Epoch 755/1000
32/32 [==============================] - 0s 125us/step - loss: 3.5044e-04 - val_loss: 1.3267e-04
Epoch 756/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4993e-04 - val_loss: 1.3245e-04
Epoch 757/1000
32/32 [==============================] - 0s 125us/step - loss: 3.4941e-04 - val_loss: 1.3223e-04
Epoch 758/1000
32/32 [==============================] - 0s 155us/step - loss: 3.4891e-04 - val_loss: 1.3199e-04
Epoch 759/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4839e-04 - val_loss: 1.3177e-04
Epoch 760/1000
32/32 [==============================] - 0s 186us/step - loss: 3.4788e-04 - val_loss: 1.3156e-04
Epoch 761/1000
32/32 [==============================] - 0s 125us/step - loss: 3.4737e-04 - val_loss: 1.3134e-04
Epoch 762/1000
32/32 [==============================] - 0s 124us/step - loss: 3.4686e-04 - val_loss: 1.3

Epoch 828/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1629e-04 - val_loss: 1.1805e-04
Epoch 829/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1586e-04 - val_loss: 1.1786e-04
Epoch 830/1000
32/32 [==============================] - 0s 155us/step - loss: 3.1544e-04 - val_loss: 1.1768e-04
Epoch 831/1000
32/32 [==============================] - 0s 155us/step - loss: 3.1502e-04 - val_loss: 1.1751e-04
Epoch 832/1000
32/32 [==============================] - 0s 157us/step - loss: 3.1460e-04 - val_loss: 1.1733e-04
Epoch 833/1000
32/32 [==============================] - 0s 155us/step - loss: 3.1418e-04 - val_loss: 1.1714e-04
Epoch 834/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1376e-04 - val_loss: 1.1697e-04
Epoch 835/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1334e-04 - val_loss: 1.1679e-04
Epoch 836/1000
32/32 [==============================] - 0s 126us/step - loss: 3.1292e-04 - val_loss: 1.1

Epoch 902/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8746e-04 - val_loss: 1.0590e-04
Epoch 903/1000
32/32 [==============================] - 0s 187us/step - loss: 2.8710e-04 - val_loss: 1.0575e-04
Epoch 904/1000
32/32 [==============================] - 0s 157us/step - loss: 2.8675e-04 - val_loss: 1.0560e-04
Epoch 905/1000
32/32 [==============================] - 0s 124us/step - loss: 2.8639e-04 - val_loss: 1.0545e-04
Epoch 906/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8603e-04 - val_loss: 1.0530e-04
Epoch 907/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8568e-04 - val_loss: 1.0515e-04
Epoch 908/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8533e-04 - val_loss: 1.0500e-04
Epoch 909/1000
32/32 [==============================] - 0s 125us/step - loss: 2.8497e-04 - val_loss: 1.0485e-04
Epoch 910/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8462e-04 - val_loss: 1.0

Epoch 976/1000
32/32 [==============================] - 0s 125us/step - loss: 2.6293e-04 - val_loss: 9.5619e-05
Epoch 977/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6263e-04 - val_loss: 9.5481e-05
Epoch 978/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6232e-04 - val_loss: 9.5364e-05
Epoch 979/1000
32/32 [==============================] - 0s 155us/step - loss: 2.6202e-04 - val_loss: 9.5226e-05
Epoch 980/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6171e-04 - val_loss: 9.5100e-05
Epoch 981/1000
32/32 [==============================] - 0s 125us/step - loss: 2.6140e-04 - val_loss: 9.4972e-05
Epoch 982/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6110e-04 - val_loss: 9.4853e-05
Epoch 983/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6079e-04 - val_loss: 9.4723e-05
Epoch 984/1000
32/32 [==============================] - 0s 155us/step - loss: 2.6049e-04 - val_loss: 9.4

[array([[ 0.34456074,  0.18667324, -0.3357817 , -0.50525033, -0.17290518],
        [-0.5969446 , -1.0845784 ,  0.17641048,  0.6343975 ,  0.8350602 ],
        [ 0.49135587, -1.0597149 , -1.053872  ,  1.0952581 ,  0.752351  ]],
       dtype=float32),
 array([-0.30958238,  0.06540466, -0.30278826, -0.34116673, -0.42009982],
       dtype=float32),
 array([[ 0.14764215,  0.33691332, -0.24587053,  0.5443069 , -0.20021267,
          0.15024915,  0.40933308,  0.46032202,  0.22687644,  0.3384165 ],
        [ 0.46556422, -0.10861446,  0.05371523,  0.07086184,  0.13462366,
         -0.24595788, -0.19713579, -0.26751637,  0.64453834,  0.10684548],
        [ 0.46351275,  0.5677999 , -0.19667733,  0.05971774,  0.40233865,
          0.49786726,  0.2194255 ,  0.4042988 , -0.15127377,  0.25692993],
        [ 0.37334844,  0.52951586,  0.30625427, -0.23887289,  0.04995485,
         -0.01299719,  0.535041  ,  0.07566474,  0.450536  , -0.2761231 ],
        [ 0.10907631,  0.2028453 , -0.4097871 ,  0.2046908

In [49]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_linear_2nd.h5')

In [50]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_relu(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_27 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_41 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_28 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 9ms/step - loss: 31.700

Epoch 71/1000
32/32 [==============================] - 0s 155us/step - loss: 0.1111 - val_loss: 0.0353
Epoch 72/1000
32/32 [==============================] - 0s 155us/step - loss: 0.1106 - val_loss: 0.0380
Epoch 73/1000
32/32 [==============================] - 0s 159us/step - loss: 0.1111 - val_loss: 0.0402
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.1118 - val_loss: 0.0413
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.1119 - val_loss: 0.0410
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.1111 - val_loss: 0.0395
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.1094 - val_loss: 0.0375
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 0.1074 - val_loss: 0.0358
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.1055 - val_loss: 0.0349
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0783 - val_loss: 0.0155
Epoch 152/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0781 - val_loss: 0.0154
Epoch 153/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0778 - val_loss: 0.0153
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0776 - val_loss: 0.0153
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0774 - val_loss: 0.0152
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0772 - val_loss: 0.0152
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0770 - val_loss: 0.0152
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0768 - val_loss: 0.0151
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0766 - val_loss: 0.0150
Epoch 160/1000
32/32 [==============================] - 0s 125us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0644 - val_loss: 0.0119
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0643 - val_loss: 0.0118
Epoch 232/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0641 - val_loss: 0.0118
Epoch 233/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0640 - val_loss: 0.0117
Epoch 234/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0638 - val_loss: 0.0117
Epoch 235/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0637 - val_loss: 0.0116
Epoch 236/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0636 - val_loss: 0.0116
Epoch 237/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0634 - val_loss: 0.0115
Epoch 238/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0633 - val_loss: 0.0115
Epoch 239/1000
32/32 [==============================] - 0s 156us

Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0544 - val_loss: 0.0088
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0543 - val_loss: 0.0088
Epoch 311/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0542 - val_loss: 0.0087
Epoch 312/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0541 - val_loss: 0.0087
Epoch 313/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0540 - val_loss: 0.0087
Epoch 314/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0539 - val_loss: 0.0086
Epoch 315/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0537 - val_loss: 0.0086
Epoch 316/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0536 - val_loss: 0.0086
Epoch 317/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0535 - val_loss: 0.0086
Epoch 318/1000
32/32 [==============================] - 0s 155us

...

Epoch 704/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0226 - val_loss: 0.0024
Epoch 705/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0226 - val_loss: 0.0024
Epoch 706/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0225 - val_loss: 0.0024
Epoch 707/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0225 - val_loss: 0.0024
Epoch 708/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0224 - val_loss: 0.0024
Epoch 709/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0223 - val_loss: 0.0024
Epoch 710/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0223 - val_loss: 0.0023
Epoch 711/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0222 - val_loss: 0.0023
Epoch 712/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0222 - val_loss: 0.0023
Epoch 713/1000
32/32 [==============================] - 0s 156us

Epoch 783/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0185 - val_loss: 0.0019
Epoch 784/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0184 - val_loss: 0.0019
Epoch 785/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0184 - val_loss: 0.0019
Epoch 786/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0183 - val_loss: 0.0019
Epoch 787/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0183 - val_loss: 0.0019
Epoch 788/1000
32/32 [==============================] - 0s 188us/step - loss: 0.0182 - val_loss: 0.0018
Epoch 789/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0182 - val_loss: 0.0018
Epoch 790/1000
32/32 [==============================] - 0s 163us/step - loss: 0.0181 - val_loss: 0.0018
Epoch 791/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0181 - val_loss: 0.0018
Epoch 792/1000
32/32 [==============================] - 0s 156us

Epoch 862/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0149 - val_loss: 0.0015
Epoch 863/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0149 - val_loss: 0.0015
Epoch 864/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0148 - val_loss: 0.0015
Epoch 865/1000
32/32 [==============================] - 0s 186us/step - loss: 0.0148 - val_loss: 0.0015
Epoch 866/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0147 - val_loss: 0.0015
Epoch 867/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0147 - val_loss: 0.0015
Epoch 868/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0147 - val_loss: 0.0015
Epoch 869/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0146 - val_loss: 0.0015
Epoch 870/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0146 - val_loss: 0.0015
Epoch 871/1000
32/32 [==============================] - 0s 126us

Epoch 941/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0119 - val_loss: 0.0012
Epoch 942/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0119 - val_loss: 0.0012
Epoch 943/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0118 - val_loss: 0.0012
Epoch 944/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0118 - val_loss: 0.0012
Epoch 945/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0118 - val_loss: 0.0011
Epoch 946/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0117 - val_loss: 0.0011
Epoch 947/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0117 - val_loss: 0.0011
Epoch 948/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0117 - val_loss: 0.0011
Epoch 949/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0116 - val_loss: 0.0011
Epoch 950/1000
32/32 [==============================] - 0s 156us

[array([[-0.42163548,  0.6059354 , -0.13677222, -0.7906114 , -1.0763352 ],
        [ 0.06679168, -0.099337  ,  0.13898559,  0.4687319 , -0.6448089 ],
        [ 1.1090984 ,  0.76668227, -1.0195371 ,  0.24008453,  0.34601167]],
       dtype=float32),
 array([0.30799362, 0.02783361, 0.66396666, 0.03773048, 0.0559358 ],
       dtype=float32),
 array([[-0.9165243 ,  0.2261486 ,  0.32673386, -0.48637864, -0.6373677 ,
         -0.8992517 ,  0.2156669 ,  0.03149367, -0.79693466, -0.24251527],
        [-0.40337026, -0.6321553 ,  0.34297574, -0.39571565, -1.0525458 ,
         -0.49060795,  0.25851688,  0.07842402, -0.20690078, -0.47556716],
        [ 0.14176187, -0.45519263, -0.37924442, -0.29292473, -0.48043913,
         -0.2172279 ,  0.5907631 ,  0.6649517 , -0.8225569 , -0.07844837],
        [-0.9881642 , -0.22664726, -0.4131226 , -1.0239978 , -0.41835934,
         -0.67992914,  0.4540248 ,  0.44805092, -0.6062681 , -0.6785382 ],
        [-0.92873305, -0.3588898 ,  0.4486165 , -0.254987  , -0

In [51]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_relu_2nd.h5')

In [52]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_sigmoid(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_29 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_44 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_30 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 10ms/step - loss: 32.33

Epoch 71/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0027 - val_loss: 0.0021
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0013
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0012 - val_loss: 7.3721e-04
Epoch 74/1000
32/32 [==============================] - 0s 187us/step - loss: 7.3555e-04 - val_loss: 3.7654e-04
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 4.6809e-04 - val_loss: 1.7965e-04
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5554e-04 - val_loss: 1.1351e-04
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 3.6444e-04 - val_loss: 1.4413e-04
Epoch 78/1000
32/32 [==============================] - 0s 187us/step - loss: 4.6070e-04 - val_loss: 2.3911e-04
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1186e-04 - val_loss: 3.6923e-04
Epoch 80/1000
32/32 [

Epoch 145/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7526e-04 - val_loss: 9.0482e-05
Epoch 146/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7439e-04 - val_loss: 9.2040e-05
Epoch 147/1000
32/32 [==============================] - 0s 187us/step - loss: 2.7361e-04 - val_loss: 9.3592e-05
Epoch 148/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7293e-04 - val_loss: 9.5090e-05
Epoch 149/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7233e-04 - val_loss: 9.6504e-05
Epoch 150/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7180e-04 - val_loss: 9.7806e-05
Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7132e-04 - val_loss: 9.8972e-05
Epoch 152/1000
32/32 [==============================] - 0s 125us/step - loss: 2.7088e-04 - val_loss: 9.9977e-05
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7047e-04 - val_loss: 1.0

Epoch 219/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4391e-04 - val_loss: 8.4300e-05
Epoch 220/1000
32/32 [==============================] - 0s 155us/step - loss: 2.4358e-04 - val_loss: 8.4132e-05
Epoch 221/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4324e-04 - val_loss: 8.3965e-05
Epoch 222/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4290e-04 - val_loss: 8.3803e-05
Epoch 223/1000
32/32 [==============================] - 0s 125us/step - loss: 2.4257e-04 - val_loss: 8.3648e-05
Epoch 224/1000
32/32 [==============================] - 0s 125us/step - loss: 2.4224e-04 - val_loss: 8.3494e-05
Epoch 225/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4191e-04 - val_loss: 8.3351e-05
Epoch 226/1000
32/32 [==============================] - 0s 125us/step - loss: 2.4158e-04 - val_loss: 8.3208e-05
Epoch 227/1000
32/32 [==============================] - 0s 125us/step - loss: 2.4125e-04 - val_loss: 8.3

Epoch 293/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2179e-04 - val_loss: 7.6598e-05
Epoch 294/1000
32/32 [==============================] - 0s 125us/step - loss: 2.2153e-04 - val_loss: 7.6517e-05
Epoch 295/1000
32/32 [==============================] - 0s 125us/step - loss: 2.2126e-04 - val_loss: 7.6428e-05
Epoch 296/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2100e-04 - val_loss: 7.6343e-05
Epoch 297/1000
32/32 [==============================] - 0s 157us/step - loss: 2.2073e-04 - val_loss: 7.6263e-05
Epoch 298/1000
32/32 [==============================] - 0s 155us/step - loss: 2.2046e-04 - val_loss: 7.6173e-05
Epoch 299/1000
32/32 [==============================] - 0s 125us/step - loss: 2.2019e-04 - val_loss: 7.6094e-05
Epoch 300/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1993e-04 - val_loss: 7.6011e-05
Epoch 301/1000
32/32 [==============================] - 0s 187us/step - loss: 2.1967e-04 - val_loss: 7.5

...

Epoch 737/1000
32/32 [==============================] - 0s 125us/step - loss: 1.3421e-04 - val_loss: 4.0761e-05
Epoch 738/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3406e-04 - val_loss: 4.0681e-05
Epoch 739/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3391e-04 - val_loss: 4.0599e-05
Epoch 740/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3376e-04 - val_loss: 4.0520e-05
Epoch 741/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3362e-04 - val_loss: 4.0441e-05
Epoch 742/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3347e-04 - val_loss: 4.0363e-05
Epoch 743/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3333e-04 - val_loss: 4.0284e-05
Epoch 744/1000
32/32 [==============================] - 0s 187us/step - loss: 1.3318e-04 - val_loss: 4.0205e-05
Epoch 745/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3303e-04 - val_loss: 4.0

Epoch 811/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2274e-04 - val_loss: 3.5106e-05
Epoch 812/1000
32/32 [==============================] - 0s 124us/step - loss: 1.2256e-04 - val_loss: 3.5029e-05
Epoch 813/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2238e-04 - val_loss: 3.4953e-05
Epoch 814/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2220e-04 - val_loss: 3.4878e-05
Epoch 815/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2203e-04 - val_loss: 3.4805e-05
Epoch 816/1000
32/32 [==============================] - 0s 155us/step - loss: 1.2185e-04 - val_loss: 3.4730e-05
Epoch 817/1000
32/32 [==============================] - 0s 125us/step - loss: 1.2168e-04 - val_loss: 3.4653e-05
Epoch 818/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2150e-04 - val_loss: 3.4577e-05
Epoch 819/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2133e-04 - val_loss: 3.4

Epoch 885/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1081e-04 - val_loss: 3.0334e-05
Epoch 886/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1067e-04 - val_loss: 3.0279e-05
Epoch 887/1000
32/32 [==============================] - 0s 125us/step - loss: 1.1052e-04 - val_loss: 3.0225e-05
Epoch 888/1000
32/32 [==============================] - 0s 155us/step - loss: 1.1038e-04 - val_loss: 3.0173e-05
Epoch 889/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1023e-04 - val_loss: 3.0120e-05
Epoch 890/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1009e-04 - val_loss: 3.0068e-05
Epoch 891/1000
32/32 [==============================] - 0s 125us/step - loss: 1.0995e-04 - val_loss: 3.0018e-05
Epoch 892/1000
32/32 [==============================] - 0s 155us/step - loss: 1.0980e-04 - val_loss: 2.9964e-05
Epoch 893/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0966e-04 - val_loss: 2.9

Epoch 959/1000
32/32 [==============================] - 0s 187us/step - loss: 1.0083e-04 - val_loss: 2.6822e-05
Epoch 960/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0070e-04 - val_loss: 2.6779e-05
Epoch 961/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0058e-04 - val_loss: 2.6737e-05
Epoch 962/1000
32/32 [==============================] - 0s 125us/step - loss: 1.0045e-04 - val_loss: 2.6699e-05
Epoch 963/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0033e-04 - val_loss: 2.6658e-05
Epoch 964/1000
32/32 [==============================] - 0s 125us/step - loss: 1.0021e-04 - val_loss: 2.6614e-05
Epoch 965/1000
32/32 [==============================] - 0s 155us/step - loss: 1.0008e-04 - val_loss: 2.6570e-05
Epoch 966/1000
32/32 [==============================] - 0s 155us/step - loss: 9.9956e-05 - val_loss: 2.6531e-05
Epoch 967/1000
32/32 [==============================] - 0s 156us/step - loss: 9.9833e-05 - val_loss: 2.6

[array([[-1.4298675 ,  0.27644145, -1.0566635 ,  0.76622355,  1.2594548 ],
        [ 1.5312579 ,  0.28291023,  0.56972045,  0.3950509 , -1.5730925 ],
        [-1.1497428 , -0.0331588 ,  1.2893088 , -1.3861564 ,  1.4841046 ]],
       dtype=float32),
 array([ 0.91897225, -0.40965942,  0.9855153 ,  0.77317923,  0.929807  ],
       dtype=float32),
 array([[ 1.5253696 , -0.9903142 ,  1.2872907 , -1.0487838 ,  1.1133215 ,
         -0.09579836, -0.63780516, -0.45755273,  1.0521785 , -1.2494339 ],
        [ 0.2478209 , -0.1601973 ,  0.6521437 , -0.43222526,  0.82868105,
         -0.5875431 , -0.43301854, -0.2727186 ,  0.8543918 , -0.19751884],
        [ 1.1773201 , -1.3230765 ,  1.5512339 , -1.25831   ,  0.8945974 ,
         -1.2736075 , -0.51824325, -0.2846994 ,  0.49638823, -0.9217735 ],
        [ 0.6837311 , -0.40257525,  1.1845977 , -0.964068  ,  1.3107548 ,
         -1.3150781 , -1.2971549 , -0.4043555 ,  0.5077204 , -0.27377632],
        [ 1.32261   , -1.044327  ,  1.5472919 , -0.5370626

In [53]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_sigmoid_2nd.h5')

In [54]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_tanh(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_31 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_47 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_32 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 10ms/step - loss: 28.93

Epoch 70/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0034 - val_loss: 0.0034
Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0043 - val_loss: 0.0038
Epoch 72/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0047 - val_loss: 0.0036
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0045 - val_loss: 0.0030
Epoch 74/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0038 - val_loss: 0.0021
Epoch 75/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0028 - val_loss: 0.0012
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 5.1754e-04
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 9.6918e-04 - val_loss: 1.1023e-04
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3422e-04 - val_loss: 3.4090e-05
Epoch 79/1000
32/32 [==============================] 

Epoch 145/1000
32/32 [==============================] - 0s 187us/step - loss: 1.8741e-04 - val_loss: 3.7963e-05
Epoch 146/1000
32/32 [==============================] - 0s 187us/step - loss: 1.8693e-04 - val_loss: 3.8158e-05
Epoch 147/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8621e-04 - val_loss: 3.8555e-05
Epoch 148/1000
32/32 [==============================] - 0s 187us/step - loss: 1.8532e-04 - val_loss: 3.9166e-05
Epoch 149/1000
32/32 [==============================] - 0s 187us/step - loss: 1.8436e-04 - val_loss: 3.9997e-05
Epoch 150/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8344e-04 - val_loss: 4.1016e-05
Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8265e-04 - val_loss: 4.2174e-05
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8202e-04 - val_loss: 4.3372e-05
Epoch 153/1000
32/32 [==============================] - 0s 187us/step - loss: 1.8154e-04 - val_loss: 4.4

Epoch 219/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5433e-04 - val_loss: 3.9359e-05
Epoch 220/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5397e-04 - val_loss: 3.9320e-05
Epoch 221/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5363e-04 - val_loss: 3.9287e-05
Epoch 222/1000
32/32 [==============================] - 0s 218us/step - loss: 1.5327e-04 - val_loss: 3.9257e-05
Epoch 223/1000
32/32 [==============================] - 0s 187us/step - loss: 1.5292e-04 - val_loss: 3.9223e-05
Epoch 224/1000
32/32 [==============================] - 0s 187us/step - loss: 1.5257e-04 - val_loss: 3.9190e-05
Epoch 225/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5223e-04 - val_loss: 3.9154e-05
Epoch 226/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5188e-04 - val_loss: 3.9111e-05
Epoch 227/1000
32/32 [==============================] - 0s 125us/step - loss: 1.5154e-04 - val_loss: 3.9

Epoch 293/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3165e-04 - val_loss: 3.5721e-05
Epoch 294/1000
32/32 [==============================] - 0s 218us/step - loss: 1.3138e-04 - val_loss: 3.5676e-05
Epoch 295/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3112e-04 - val_loss: 3.5634e-05
Epoch 296/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3086e-04 - val_loss: 3.5591e-05
Epoch 297/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3060e-04 - val_loss: 3.5548e-05
Epoch 298/1000
32/32 [==============================] - 0s 187us/step - loss: 1.3033e-04 - val_loss: 3.5505e-05
Epoch 299/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3007e-04 - val_loss: 3.5464e-05
Epoch 300/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2982e-04 - val_loss: 3.5421e-05
Epoch 301/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2956e-04 - val_loss: 3.5

...

Epoch 737/1000
32/32 [==============================] - 0s 156us/step - loss: 6.7492e-05 - val_loss: 2.7316e-05
Epoch 738/1000
32/32 [==============================] - 0s 156us/step - loss: 6.7418e-05 - val_loss: 2.7311e-05
Epoch 739/1000
32/32 [==============================] - 0s 125us/step - loss: 6.7341e-05 - val_loss: 2.7305e-05
Epoch 740/1000
32/32 [==============================] - 0s 125us/step - loss: 6.7267e-05 - val_loss: 2.7300e-05
Epoch 741/1000
32/32 [==============================] - 0s 155us/step - loss: 6.7191e-05 - val_loss: 2.7294e-05
Epoch 742/1000
32/32 [==============================] - 0s 125us/step - loss: 6.7119e-05 - val_loss: 2.7286e-05
Epoch 743/1000
32/32 [==============================] - 0s 156us/step - loss: 6.7044e-05 - val_loss: 2.7282e-05
Epoch 744/1000
32/32 [==============================] - 0s 125us/step - loss: 6.6968e-05 - val_loss: 2.7280e-05
Epoch 745/1000
32/32 [==============================] - 0s 155us/step - loss: 6.6895e-05 - val_loss: 2.7

Epoch 811/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2353e-05 - val_loss: 2.6988e-05
Epoch 812/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2289e-05 - val_loss: 2.6985e-05
Epoch 813/1000
32/32 [==============================] - 0s 187us/step - loss: 6.2225e-05 - val_loss: 2.6982e-05
Epoch 814/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2163e-05 - val_loss: 2.6979e-05
Epoch 815/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2098e-05 - val_loss: 2.6975e-05
Epoch 816/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2035e-05 - val_loss: 2.6972e-05
Epoch 817/1000
32/32 [==============================] - 0s 154us/step - loss: 6.1973e-05 - val_loss: 2.6967e-05
Epoch 818/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1909e-05 - val_loss: 2.6968e-05
Epoch 819/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1845e-05 - val_loss: 2.6

Epoch 885/1000
32/32 [==============================] - 0s 156us/step - loss: 5.7981e-05 - val_loss: 2.6794e-05
Epoch 886/1000
32/32 [==============================] - 0s 187us/step - loss: 5.7927e-05 - val_loss: 2.6792e-05
Epoch 887/1000
32/32 [==============================] - 0s 156us/step - loss: 5.7872e-05 - val_loss: 2.6788e-05
Epoch 888/1000
32/32 [==============================] - 0s 156us/step - loss: 5.7816e-05 - val_loss: 2.6787e-05
Epoch 889/1000
32/32 [==============================] - 0s 187us/step - loss: 5.7763e-05 - val_loss: 2.6786e-05
Epoch 890/1000
32/32 [==============================] - 0s 156us/step - loss: 5.7709e-05 - val_loss: 2.6784e-05
Epoch 891/1000
32/32 [==============================] - 0s 156us/step - loss: 5.7654e-05 - val_loss: 2.6783e-05
Epoch 892/1000
32/32 [==============================] - 0s 156us/step - loss: 5.7601e-05 - val_loss: 2.6782e-05
Epoch 893/1000
32/32 [==============================] - 0s 156us/step - loss: 5.7548e-05 - val_loss: 2.6

Epoch 959/1000
32/32 [==============================] - 0s 156us/step - loss: 5.4217e-05 - val_loss: 2.6700e-05
Epoch 960/1000
32/32 [==============================] - 0s 125us/step - loss: 5.4172e-05 - val_loss: 2.6702e-05
Epoch 961/1000
32/32 [==============================] - 0s 125us/step - loss: 5.4125e-05 - val_loss: 2.6700e-05
Epoch 962/1000
32/32 [==============================] - 0s 156us/step - loss: 5.4075e-05 - val_loss: 2.6700e-05
Epoch 963/1000
32/32 [==============================] - 0s 125us/step - loss: 5.4028e-05 - val_loss: 2.6699e-05
Epoch 964/1000
32/32 [==============================] - 0s 156us/step - loss: 5.3981e-05 - val_loss: 2.6698e-05
Epoch 965/1000
32/32 [==============================] - 0s 156us/step - loss: 5.3936e-05 - val_loss: 2.6695e-05
Epoch 966/1000
32/32 [==============================] - 0s 125us/step - loss: 5.3890e-05 - val_loss: 2.6698e-05
Epoch 967/1000
32/32 [==============================] - 0s 125us/step - loss: 5.3842e-05 - val_loss: 2.6

[array([[ 0.4418993 , -0.9924778 ,  1.4855046 , -0.43957102, -0.5873704 ],
        [-0.45432314,  1.2622943 , -1.1151009 , -1.2847834 ,  1.2202035 ],
        [ 0.62762415,  1.3228725 , -1.1606499 , -1.1440914 , -1.077212  ]],
       dtype=float32),
 array([-0.05569606,  0.7148598 ,  0.6447647 ,  0.68643385,  0.7162877 ],
       dtype=float32),
 array([[ 1.0668627 , -0.5749267 ,  0.37502658,  0.49871224,  0.22313769,
          0.19043618,  1.0259385 , -1.0016663 , -0.38766634,  0.66892064],
        [ 0.97462857, -1.2066432 ,  0.29968235,  0.69233644,  1.2295597 ,
          0.03709182,  0.5549572 , -1.3321874 , -0.12321477,  0.9630039 ],
        [ 0.07031487, -0.5737888 ,  0.12287904,  1.2244643 ,  1.1991793 ,
          0.8922235 ,  0.9619157 , -1.0768243 , -1.1684618 ,  0.39470005],
        [ 1.1648592 , -1.0305548 ,  1.1416585 ,  0.02360647,  0.9867999 ,
          1.0824572 ,  1.1180915 , -0.1762995 , -1.2231485 ,  0.43959427],
        [ 0.9803563 , -1.2821087 ,  0.7161241 ,  1.3432412

In [55]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_tanh_2nd.h5')